Copyright 2020 – Tyler Ngo<br />
License - This work is licensed under MIT license - https://opensource.org/licenses/MIT<br />  
PREAMBLE<br />
This is a tutorial, written in python, to demonstrate how to solve a simple linear regression statistical problem.
By using IPython's [rich display system](https://nbviewer.jupyter.org/github/ipython/ipython/blob/2.x/examples/Notebook/Display%20System.ipynb) capability, we also hope to turn it into a how-to document, saving us from the need to reproduce a Word Doc to serve the same purpose.<br />  
In this example, we aim to assess TBFM-reported delays for flights heading to the meter fix ORD_DOOGE before and after APREQ prescheduling to ORD was implemented at CLT in June of 2019.<br />  
TODO:<br />
+ Annotate each cell to summarize what it's trying to do; make sure to explain the test statistics and the effect size<br />
    Esp. for the effect size, I'd like to see a statement like this: "an effect size of 0.8 means that the average delay after APREQ prescheduling is 0.8 standard deviations above the average delay before APREQ prescheduling. Alternatively, we can state that the average delay after APREQ prescheduling is higher than 79% of the delays observed before APREQ prescheduling. An effect size of .8 is categorized as large." This example statement clearly makes the assumption that the delay distribution is normally distributed. Review [this page](https://www.leeds.ac.uk/educol/documents/00002182.htm) to see how effect size can be interpreted.<br />
+ Include the math formula for each test statistic (look into the math library and latex - see the link for rich display system above). We want this displayed to explain to other analysts when/how to use a test statistic.<br />
+ Verify that the effect size is correctly computed within their function calls (I made 2: Cohen's D for normal dist. & Cliff's delta for non-parametric).<br />
+ Ensure data are parsed correctly and that I'm picking the correct subset for each purpose<br />
+ See if you can merge the separate plots (histograms, density, & boxplots) for each airport into one page; that would leave one quadrant for the test statistics, the effect size, and a short explanation of what they mean. Do the same for the streamclass.<br />
+ See if you can fix some image texts, etc; goal is to make them look uniform, texts are reasonably located within the charts.<br />

PROBLEM STATEMENT<br />
In this example, we seek to address 2 questions:<br />
1. Is there a statistically significant difference in delay distributions before and after APREQ prescheduling was implemented in June 2019?<br />
2. Is there a linear relationship between TBFM reported delay and the associated flight's distance to the meter fix from its departure airport?<br />  

METHODS:<br />
For the 1st question, we will assess overall delay for the ORD_DOOGE streamclass and delay for each airport going to ORD_DOOGE. We perform the following steps for each airport and overall streamclass delay before and after APREQ prescheduling:<br />
1. Compute descriptive statistics<br />
2. Plot delay distributions to assess location, scale, and shape<br />
3. Assess outliers for each set of data. Here, I provided 2 methods. However, we can choose a more rigorous method to assess whether to remove or incorporate outliers if we wish.<br />
    + Remove outliers using Z method (retains 99% of data)<br />
    + Remove outliers using IQR method - lower cutoff=(25th percentile - 150% of IQR); upper cutoff=(75th percentile + 150% of IQR)<br />
4. Assess the pdf of the delay distribution, then test for goodness-of-fit.  Here, I attempted 2 methods:<br />
    + Fit a # of pdfs, select one with lowest SSE, then test for goodness of fit using dist parameters.<br />
        - Use KS test: split data into train and test set; use train set to find dist params, use test set for goodness-of-fit.<br />
        - You can also use Lilliefors test if you're testing against a normal or exponential dist. You don't need to split the dataset if you use Lilliefors since it also estimates the distribution parameters.<br />
    + Use MLE starting with the dist and dist params from SSE computation to find a better dist parameters, then test for goodness-of-fit using this dist parameters. <font color='red'>This section of code needs some TLC.</font><br />
5. Try to normalize the delay based on the knowledge we obtain from step 4. Step 4 used to be very important because in order to attempt to normalize, we have to know the pdf.  It doesn't seem to be as important now.  But step 4 is stil important in gaining insight and understanding of the data.<br />
6. Test for normality.<br />
    + Use KS test: split data into train and test set; use train set to find dist params, use test set for goodness-of-fit.<br />
    + Or use Lilliefors test.<br />
    + Just for fun, I've also added the Shapiro-Wilk test statistic to compare with the other 2. This one generally works better for larger sample sizes.<br /> 
7. Test for statistically significant difference between delay dist before and after APREQ prescheduling. Depending on whether normality test passes or not, we decide which test statistics to use.<br />
    + If passes for normality, use Welch's-T test.<br />
    + If test for normality does not pass, use KS test.<br />
8. If a statistically significant difference is found, then compute effect size. Interpret the effect size to assess whether the difference was important. Here I provide 2 methods. We can investigate other methods to compute effect size.<br />
    + Cohen's D, use if normality test is passed<br />
    + Cliff's Delta, use for non-parametric.<br />  

--------------------------------------------------------------------------<br />

For the 2nd question, we will assess whether there is a linear relationship between the flight's TBFM-reported delay and its distance to the meter fix from its departure airport. We will perform the following steps:<br />
1. Compute descriptive statistics<br />
2. Plot delay and meterFixDistance distribution to assess location, scale, and shape<br />
3. Assess outliers for each. See step 3 above.<br />
4. Assess pdf for each. See step 4 above.<br />
5. Try to normalize the datasets<br />
6. Test for normality. Attempting to fit a linear regression is usually performed only if normality is passed.<br />
7. Run linear regression model and test for goodness-of-fit using pearson's R.<br />
7. If the pearson's R looks reasonable (here, we have to decide what is reasonable) and a linear relationship is found, then we should perform more rigorous tesing of assumptions as listed here:<br />
    + Linearity<br />
    + Homoscedasticity (Constant Variance)<br />
    + Normality<br />
    + Independence of Error<br />
    + Outlier and Influential Observation Analysis<br />



In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf
import seaborn as sns

from sklearn import preprocessing
from sklearn.preprocessing import power_transform
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import scipy as scp
import scipy.stats as st
from scipy.optimize import minimize
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel
from statsmodels.stats.diagnostic import lilliefors

import os
import glob
import datetime
from io import StringIO
from datetime import datetime
import warnings
from math import log


In [2]:
######################################################################
#Define all function calls before main                               #
######################################################################

#######################################################
#function calls for finding best pdf by SSE comparison
#######################################################

def best_fit_distribution(data, bins=200, ax=None):
    """Model data by finding best fit distribution to data"""
    # Get histogram of original data
    y, x = np.histogram(data, bins=bins, density=True)
    x = (x + np.roll(x, -1))[:-1] / 2.0

    # Distributions to check
    """
    DISTRIBUTIONS = [        
        st.alpha,st.anglit,st.arcsine,st.beta,st.betaprime,st.bradford,st.burr,st.cauchy,st.chi,st.chi2,st.cosine,
        st.dgamma,st.dweibull,st.erlang,st.expon,st.exponnorm,st.exponweib,st.exponpow,st.f,st.fatiguelife,st.fisk,
        st.foldcauchy,st.foldnorm,st.frechet_r,st.frechet_l,st.genlogistic,st.genpareto,st.gennorm,st.genexpon,
        st.genextreme,st.gausshyper,st.gamma,st.gengamma,st.genhalflogistic,st.gilbrat,st.gompertz,st.gumbel_r,
        st.gumbel_l,st.halfcauchy,st.halflogistic,st.halfnorm,st.halfgennorm,st.hypsecant,st.invgamma,st.invgauss,
        st.invweibull,st.johnsonsb,st.johnsonsu,st.ksone,st.kstwobign,st.laplace,st.levy,st.levy_l,st.levy_stable,
        st.logistic,st.loggamma,st.loglaplace,st.lognorm,st.lomax,st.maxwell,st.mielke,st.nakagami,st.ncx2,st.ncf,
        st.nct,st.norm,st.pareto,st.pearson3,st.powerlaw,st.powerlognorm,st.powernorm,st.rdist,st.reciprocal,
        st.rayleigh,st.rice,st.recipinvgauss,st.semicircular,st.t,st.triang,st.truncexpon,st.truncnorm,st.tukeylambda,
        st.uniform,st.vonmises,st.vonmises_line,st.wald,st.weibull_min,st.weibull_max,st.wrapcauchy
    ]
    """

    DISTRIBUTIONS = [        
        'dgamma','dweibull','expon','exponnorm','exponweib','exponpow','foldnorm','gamma','gengamma',
        'halfnorm','halfgennorm','invgamma','invweibull','loggamma','lognorm','norm','powerlognorm',
        'powernorm','weibull_min','weibull_max']

    # Best holders
    best_distribution = st.norm
    best_params = (0.0, 1.0)
    best_sse = np.inf

    # Estimate distribution parameters from data
    for dist_name in DISTRIBUTIONS:

        # Try to fit the distribution
        try:
            # Ignore warnings from data that can't be fit
            with warnings.catch_warnings():
                warnings.filterwarnings('ignore')

                # fit dist to data
                distribution = getattr(st, dist_name)
                if "weibull" in dist_name:
                    params = distribution.fit(data.iloc[:,0], floc=0, f0=1) #have to convert dataframe to series
                elif "gamma" in dist_name:
                    params = distribution.fit(data, floc=0)
                elif "lognorm" in dist_name:
                    params = distribution.fit(data, floc=0)
                else:
                    params = distribution.fit(data)
                print(dist_name, " distribution parameters: ", params)
                

                # Separate parts of parameters
                arg = params[:-2]
                loc = params[-2]
                scale = params[-1]

                # Calculate fitted PDF and error with fit in distribution
                pdf = distribution.pdf(x, loc=loc, scale=scale, *arg)
                sse = np.sum(np.power(y - pdf, 2.0))
                print("SSE for ", dist_name, ": ", sse)

                # if axis pass in add to plot
                try:
                    if ax:
                        pd.Series(pdf, x).plot(ax=ax)
                    end
                except Exception:
                    pass

                # identify if this distribution is better
                if best_sse > sse > 0:
                    best_distribution = distribution
                    best_params = params
                    best_sse = sse

        except Exception:
            pass

    return (best_distribution.name, best_params)

def make_pdf(dist, params, size=10000):
    """Generate distributions's Probability Distribution Function """

    # Separate parts of parameters
    arg = params[:-2]
    loc = params[-2]
    scale = params[-1]

    # Get sane start and end points of distribution
    start = dist.ppf(0.01, *arg, loc=loc, scale=scale) if arg else dist.ppf(0.01, loc=loc, scale=scale)
    end = dist.ppf(0.99, *arg, loc=loc, scale=scale) if arg else dist.ppf(0.99, loc=loc, scale=scale)

    # Build PDF and turn into pandas Series
    x = np.linspace(start, end, size)
    y = dist.pdf(x, loc=loc, scale=scale, *arg)
    pdf = pd.Series(y, x)

    return pdf

############################################################
#function call to compute cohen's d for independent samples
############################################################
#for an overview of effect size computation: https://www.leeds.ac.uk/educol/documents/00002182.htm
#Pooled standard deviation is used to account for difference in variation between the 2 samples
#However, the use of a pooled estimate of standard deviation depends
#on the assumption that the two calculated standard deviations are estimates of the same population value
def cohend(d1, d2):
    # calculate the size of samples
    n1, n2 = len(d1), len(d2)
    # calculate the variance of the samples
    s1, s2 = np.var(d1, ddof=1), np.var(d2, ddof=1)
    # calculate the pooled standard deviation
    s = np.sqrt(((n1 - 1) * s1 + (n2 - 1) * s2) / (n1 + n2 - 2))
    # calculate the means of the samples
    u1, u2 = np.mean(d1), np.mean(d2)
    # calculate the effect size
    return (u1 - u2) / s

################################################################
#function calls to compute cliff's delta for independent samples
################################################################
#for another overview of effect size: http://www.maths.dur.ac.uk/~dma0je/PG/TrainingWeek10/RSC-2010.pdf
def cliffsDelta(lst1, lst2, **dull):
#this function def is downloaded from https://github.com/neilernst/cliffsDelta; need to verify for correctness
    """Returns delta and true if there are more than 'dull' differences"""
    if not dull:
        dull = {'small': 0.147, 'medium': 0.33, 'large': 0.474} # effect sizes from (Hess and Kromrey, 2004)
    m, n = len(lst1), len(lst2)
    lst2 = sorted(lst2)
    j = more = less = 0
    for repeats, x in runs(sorted(lst1)):
        while j <= (n - 1) and lst2[j] < x:
            j += 1
        more += j*repeats
        while j <= (n - 1) and lst2[j] == x:
            j += 1
        less += (n - j)*repeats
    d = (more - less) / (m*n)
    size = lookup_size(d, dull)
    return d, size


def lookup_size(delta: float, dull: dict) -> str:
    """
    :type delta: float
    :type dull: dict, a dictionary of small, medium, large thresholds.
    """
    delta = abs(delta)
    if delta < dull['small']:
        return 'negligible'
    if dull['small'] <= delta < dull['medium']:
        return 'small'
    if dull['medium'] <= delta < dull['large']:
        return 'medium'
    if delta >= dull['large']:
        return 'large'


def runs(lst):
    """Iterator, chunks repeated values"""
    for j, two in enumerate(lst):
        if j == 0:
            one, i = two, 0
        if one != two:
            yield j - i, one
            i = j
        one = two
    yield j - i + 1, two

############################################################
#function call to identify outliers using Z-Score method
############################################################

#The intuition behind Z-score is to describe any data point by finding their relationship 
#with the Standard Deviation and Mean of the group of data points.
#Z-score is finding the distribution of data where mean is 0 and standard deviation is 1 i.e. normal distribution.
#while calculating the Z-score we re-scale and center the data and look for data points which are too far from zero.
#These data points which are way too far from zero will be treated as the outliers.
#In most of the cases a threshold of 3 or -3 is used
#i.e if the Z-score value is greater than or less than 3 or -3 respectively, that data point will be identified as outliers.
#99% of the data is retained(6 sigma = three standard deviations above and below the mean)

def outliers_Z(d):
    # calculate summary statistics
    data_mean, data_std = np.mean(d), np.std(d)
    # identify outliers
    cut_off = data_std * 3
    lower, upper = data_mean - cut_off, data_mean + cut_off
    # identify outliers
    outliers = [x for x in d if x < lower or x > upper]
    print('Number of outliers: %d' % len(outliers))

    # remove outliers
    removed_outliers = [x for x in d if x >= lower and x <= upper]
    print('No-outlier observations for dataset: %d' % len(removed_outliers))
    return removed_outliers

############################################################
#function call to identify outliers using IQR method
############################################################

#It is a measure of the dispersion similar to standard deviation or variance, but is much more robust against outliers.
#IQR is the middle 50% = Q3-Q1 
#lower cutoff=(25th percentile - 150% of IQR); upper cutoff=(75th percentile + 150% of IQR)

def outliers_IQR(d):    
    # calculate interquartile range
    q25, q75 = np.percentile(d, 25), np.percentile(d, 75)
    iqr = q75 - q25
    print('Percentiles for dataset: 25th=%.3f, 75th=%.3f, IQR=%.3f' % (q25, q75, iqr))
    # calculate the outlier cutoff
    cut_off = iqr * 1.5
    lower, upper = q25 - cut_off, q75 + cut_off
    # identify outliers
    outliers = [x for x in d if x < lower or x > upper]
    print('Number of outliers: %d' % len(outliers))

    # remove outliers using IQR
    removed_outliers = [x for x in d if x >= lower and x <= upper]
    print('No-outlier observations for dataset: %d' % len(removed_outliers))
    return removed_outliers

############################################################
#function call to compute aic for regression
############################################################

def calculate_aic(n, mse, num_params):
    aic = n * log(mse) + 2 * num_params
    return aic

############################################################
#function call to compute bic for regression
############################################################

def calculate_bic(n, mse, num_params):
    bic = n * log(mse) + num_params * log(n)
    return bic

#######################################################################
#function call to normalize data using scaler & power transform
#######################################################################

##################
#Quoted from part 2 section 16 of: http://www.faqs.org/faqs/ai-faq/neural-nets/part1/preamble.html
#"Rescaling" a vector means to add or subtract a
#constant and then multiply or divide by a constant, as you would do to
#change the units of measurement of the data, for example, to convert a
#temperature from Celsius to Fahrenheit. 

#"Normalizing" a vector most often means dividing by a norm of the vector,
#for example, to make the Euclidean length of the vector equal to one. In the
#NN literature, "normalizing" also often refers to rescaling by the minimum
#and range of the vector, to make all the elements lie between 0 and 1. 

#"Standardizing" a vector most often means subtracting a measure of location
#and dividing by a measure of scale. For example, if the vector contains
#random values with a Gaussian distribution, you might subtract the mean and
#divide by the standard deviation, thereby obtaining a "standard normal"
#random variable with mean 0 and standard deviation 1. 

#However, all of the above terms are used more or less interchangeably
#depending on the customs within various fields.
##################

##################
#to learn more: https://scikit-learn.org/stable/auto_examples/preprocessing/plot_map_data_to_normal.html
#https://scikit-learn.org/stable/modules/preprocessing.html

#You can also consider the following transform functions available on scikit-learn:
#mm_scaler = preprocessing.MinMaxScaler()
#data = mm_scaler.fit_transform(d)

#scaler = preprocessing.StandardScaler()
#data = scaler.fit_transform(d)

#transformer = preprocessing.RobustScaler()
#data = transformer.fit_transform(d)

#max_abs_scaler = preprocessing.MaxAbsScaler()
#data = max_abs_scaler.fit_transform(pd.DataFrame(d))

#power_xformer = preprocessing.PowerTransformer()
#data_xformed = power_xformer.fit(data).transform(data)

#data_xformed = preprocessing.power_transform(data, method='yeo-johnson')

#data_xformed, lmbda = st.yeojohnson(data)

#posdata = data[data > 0] #boxcox test uses strictly positive data only
#data_xformed = st.boxcox(posdata, alpha)
#################

def xform(d):
    data_xformed = preprocessing.power_transform(d, method='box-cox')
    return data_xformed


In [3]:
#Read in data by ARTCC
build_data = True
TBFM_path = '<data directory here>'
if build_data: 
    indiv_files = []
    allFiles = glob.glob(os.path.join(TBFM_path, '*ZTL*.csv'),recursive=True)
    allFiles = np.sort(allFiles)
    for f in allFiles:
        #print(f)
        df = pd.read_csv(f, delim_whitespace=False, dtype={'acid': str, 'streamClass': str, 'FinalDelay': float, 'meterFixDistance': float})
        df = df.dropna(axis=0, subset=['acid', 'streamClass', 'FinalDelay', 'meterFixDistance', 'coordinationTimeZulu'])
        df = df[(df['FinalDelay'] != 0)]
        indiv_files.append(df)
     
    df_ZTL = pd.concat(indiv_files)

    #format specific fields with data type as int
    df_ZTL['FinalDelay'] = df_ZTL['FinalDelay'].astype(int)
    df_ZTL['meterFixDistance'] = df_ZTL['meterFixDistance'].astype(float)
    df_ZTL['meterFixDistance'] = df_ZTL['meterFixDistance'].abs()
    df_ZTL['coordinationTimeZulu'] = df_ZTL['coordinationTimeZulu'].astype(str)
    df_ZTL.sort_values(by = 'streamClass')
    
    df_ZTL.to_csv('Dataset_ZTL.csv')
    print("ZTL dataset.shape: ", df_ZTL.shape)


ZTL dataset.shape:  (172423, 24)


In [4]:
#Compute stats for delay per streamclass
subset_delay = df_ZTL[['origApt', 'streamClass', 'FinalDelay', 'coordinationTimeZulu']]
strClass_delay = pd.melt(subset_delay, id_vars='streamClass', value_vars='FinalDelay')
strClass_delay_pivot = strClass_delay.pivot(columns='streamClass', values='value')
delay_ZTL = strClass_delay_pivot.describe()
delay_ZTL.transpose().to_csv('Delay_by_Streamclass_Statistic_ZTL.csv')

#Compute stats for distance to meter fix per streamclass
subset_dist = df_ZTL[['origApt', 'streamClass', 'meterFixDistance', 'coordinationTimeZulu']]
strClass_dist = pd.melt(subset_dist, id_vars='streamClass', value_vars='meterFixDistance')
strClass_dist_pivot = strClass_dist.pivot(columns='streamClass', values='value')
dist_ZTL = strClass_dist_pivot.describe()
dist_ZTL.transpose().to_csv('meterFixDist_by_Streamclass_Statistic_ZTL.csv')

#We will pick ORD_DOOGE for this example
subset_ORDDOOGE = df_ZTL[df_ZTL['streamClass']=='ORD_DOOGE']


################################################################################
#If you want to set the median value of meterFixDistance from each airport     #
#set the median value for each airport here, convert this cell type to "code"  #
################################################################################

subset_ORDDOOGE.loc[subset_ORDDOOGE['origApt'] == 'AGS', ['meterFixDistance']] = 223
subset_ORDDOOGE.loc[subset_ORDDOOGE['origApt'] == 'ATL', ['meterFixDistance']] = 295
subset_ORDDOOGE.loc[subset_ORDDOOGE['origApt'] == 'AVL', ['meterFixDistance']] = 94
subset_ORDDOOGE.loc[subset_ORDDOOGE['origApt'] == 'BHM', ['meterFixDistance']] = 347
subset_ORDDOOGE.loc[subset_ORDDOOGE['origApt'] == 'CAE', ['meterFixDistance']] = 215.1
subset_ORDDOOGE.loc[subset_ORDDOOGE['origApt'] == 'CHS', ['meterFixDistance']] = 275
subset_ORDDOOGE.loc[subset_ORDDOOGE['origApt'] == 'CLT', ['meterFixDistance']] = 142.6
subset_ORDDOOGE.loc[subset_ORDDOOGE['origApt'] == 'GSO', ['meterFixDistance']] = 140
subset_ORDDOOGE.loc[subset_ORDDOOGE['origApt'] == 'GSP', ['meterFixDistance']] = 139.3
subset_ORDDOOGE.loc[subset_ORDDOOGE['origApt'] == 'GAI', ['meterFixDistance']] = 161
subset_ORDDOOGE.loc[subset_ORDDOOGE['origApt'] == 'GSO', ['meterFixDistance']] = 390
subset_ORDDOOGE.loc[subset_ORDDOOGE['origApt'] == 'JAX', ['meterFixDistance']] = 400
subset_ORDDOOGE.loc[subset_ORDDOOGE['origApt'] == 'MCO', ['meterFixDistance']] = 520
subset_ORDDOOGE.loc[subset_ORDDOOGE['origApt'] == 'MYR', ['meterFixDistance']] = 267
subset_ORDDOOGE.loc[subset_ORDDOOGE['origApt'] == 'RDU', ['meterFixDistance']] = 214.3
subset_ORDDOOGE.loc[subset_ORDDOOGE['origApt'] == 'SAV', ['meterFixDistance']] = 310


In [5]:
#Clean data for streamclasses plots

#drop stream classes with less than 10 counts for ZTL center, run once only or restart kernel
#it'll throw errors if it can't find these streamclasses in the data
strClass_delay_pivot = strClass_delay_pivot.drop(columns= [
    'BWI_TONIO',
    'DTW_IIU',
    'EWR_J43_ROD',
    'E_DTW_SUBWY',
    'FLL_JEFOI',
    'FLL_Q75',
    'IAD_J83_APE',
    'LGA_TONIO',
    'MEM_MATCN',
    'MIA_JEFOI',
    'MLLET_RASLN_NAVEE_JET',
    'MSP_BNA',
    'TEB_PSK',
    'TEB_TWINS',
    'TPA_ACORI',
    'BOS_Q22',
    'DAL_PUDJE',
    'DCA_J83_APE',
    'LGA_TWINS',
    'MCO_JEFOI',
    'MMU_Q22',
    'OTHR_AG1',
    'SPARE_13',
    'DAL_MEM',
    'DTW_JAMOX',
    'HOU_MEI',
    'IAD_TWINS',
    'JFK_J43_ROD',
    'MCO_AREA4_PIE',
    'OTHR_DN1',
    'TPA_JEFOI',
    'W_DTW_SUBWY',
    'IAH_MEI',
    'IAH_Q40_MEI',
    'SPARE_6',
    'IAH_MEM',
    'LGA_J43_ROD',
    'RSW_HONID',
    'DCA_TONIO',
    'MCO_Q75',
    'OTHR_MHZ',
    'OTHR_SJI',
    'MIA_Q75',
    'ROME_TURBO_PROP',
    'OTHR_MGM',
    'ROME_PISTON',
    'TPA_HONID'])

strClass_dist_pivot = strClass_dist_pivot.drop(columns= [
    'BWI_TONIO',
    'DTW_IIU',
    'EWR_J43_ROD',
    'E_DTW_SUBWY',
    'FLL_JEFOI',
    'FLL_Q75',
    'IAD_J83_APE',
    'LGA_TONIO',
    'MEM_MATCN',
    'MIA_JEFOI',
    'MLLET_RASLN_NAVEE_JET',
    'MSP_BNA',
    'TEB_PSK',
    'TEB_TWINS',
    'TPA_ACORI',
    'BOS_Q22',
    'DAL_PUDJE',
    'DCA_J83_APE',
    'LGA_TWINS',
    'MCO_JEFOI',
    'MMU_Q22',
    'OTHR_AG1',
    'SPARE_13',
    'DAL_MEM',
    'DTW_JAMOX',
    'HOU_MEI',
    'IAD_TWINS',
    'JFK_J43_ROD',
    'MCO_AREA4_PIE',
    'OTHR_DN1',
    'TPA_JEFOI',
    'W_DTW_SUBWY',
    'IAH_MEI',
    'IAH_Q40_MEI',
    'SPARE_6',
    'IAH_MEM',
    'LGA_J43_ROD',
    'RSW_HONID',
    'DCA_TONIO',
    'MCO_Q75',
    'OTHR_MHZ',
    'OTHR_SJI',
    'MIA_Q75',
    'ROME_TURBO_PROP',
    'OTHR_MGM',
    'ROME_PISTON',
    'TPA_HONID'])


In [6]:
#Split data into before and after APREQ prescheduling to ORD was turned on in June 2019

ORD_preAPREQ = datetime.strptime("20190601 00:00:00", "%Y%m%d %H:%M:%S")
ORD_pstAPREQ = datetime.strptime("20190701 00:00:00", "%Y%m%d %H:%M:%S")

subset_ORDDOOGE_preAPREQ = subset_ORDDOOGE[subset_ORDDOOGE['coordinationTimeZulu'].apply(lambda x: datetime.strptime(x, '%Y%m%d %H:%M:%S')) < ORD_preAPREQ]
subset_ORDDOOGE_pstAPREQ = subset_ORDDOOGE[subset_ORDDOOGE['coordinationTimeZulu'].apply(lambda x: datetime.strptime(x, '%Y%m%d %H:%M:%S')) >= ORD_pstAPREQ]


In [7]:
#Compute stats for delay per airport & streamclass before & after APREQ prescheduling to ORD
print("ORD_DOOGE delay statistics before APREQ Prescheduling:\n", subset_ORDDOOGE_preAPREQ['FinalDelay'].describe())
print("\n")
print("ORD_DOOGE delay statistics after APREQ Prescheduling:\n", subset_ORDDOOGE_pstAPREQ['FinalDelay'].describe())

origApt_delay_ORDDOOGE_preAPREQ = pd.melt(subset_ORDDOOGE_preAPREQ, id_vars='origApt', value_vars='FinalDelay')
origApt_delay_ORDDOOGE_preAPREQ_pivot = origApt_delay_ORDDOOGE_preAPREQ.pivot(columns='origApt', values='value')
origApt_delay_ORDDOOGE_preAPREQ_pivot = origApt_delay_ORDDOOGE_preAPREQ_pivot.loc[:, origApt_delay_ORDDOOGE_preAPREQ_pivot.columns.notnull()]
delay_ORDDOOGE_preAPREQ = origApt_delay_ORDDOOGE_preAPREQ_pivot.describe()
delay_ORDDOOGE_preAPREQ.transpose().to_csv('Delay_by_Airports_to_ORD_DOOGE_preAPREQ_statistics.csv')

origApt_delay_ORDDOOGE_pstAPREQ = pd.melt(subset_ORDDOOGE_pstAPREQ, id_vars='origApt', value_vars='FinalDelay')
origApt_delay_ORDDOOGE_pstAPREQ_pivot = origApt_delay_ORDDOOGE_pstAPREQ.pivot(columns='origApt', values='value')
origApt_delay_ORDDOOGE_pstAPREQ_pivot = origApt_delay_ORDDOOGE_pstAPREQ_pivot.loc[:, origApt_delay_ORDDOOGE_pstAPREQ_pivot.columns.notnull()]
delay_ORDDOOGE_pstAPREQ = origApt_delay_ORDDOOGE_pstAPREQ_pivot.describe()
delay_ORDDOOGE_pstAPREQ.transpose().to_csv('Delay_by_Airports_to_ORD_DOOGE_pstAPREQ_statistics.csv')


ORD_DOOGE delay statistics before APREQ Prescheduling:
 count    1754.000000
mean       12.692702
std        10.528623
min         1.000000
25%         5.000000
50%         9.000000
75%        18.000000
max        61.000000
Name: FinalDelay, dtype: float64


ORD_DOOGE delay statistics after APREQ Prescheduling:
 count    2465.000000
mean       13.335091
std        11.057392
min         1.000000
25%         5.000000
50%        10.000000
75%        18.000000
max        86.000000
Name: FinalDelay, dtype: float64


In [8]:
###################################################################
#Plot delays and meterFixDistance for streamclasses in the ARTCC  #
###################################################################

pdf = matplotlib.backends.backend_pdf.PdfPages("ZTL_plots.pdf")

# histograms
strClass_delay_pivot.hist(sharex=False, sharey=False, xlabelsize=.2, ylabelsize=.2, figsize=(40, 24), bins=50)
plt.suptitle("Histograms - Delays - ZTL center", fontsize=50)
pdf.savefig()

strClass_dist_pivot.hist(sharex=False, sharey=False, xlabelsize=.2, ylabelsize=.2, figsize=(40, 24), bins=50)
plt.suptitle("Histograms - Meter Fix Distance - ZTL center", fontsize=50)
pdf.savefig()

# density
strClass_delay_pivot.plot(kind='density', subplots=True, figsize=(40,24), layout=(10,10), sharex=False, sharey=False, legend=False, fontsize=.2)
plt.suptitle("KDE - Delays - ZTL center", fontsize=50)
pdf.savefig()

strClass_dist_pivot.plot(kind='density', subplots=True, figsize=(40,24), layout=(10,10), sharex=False, sharey=False, legend=False, fontsize=.2)
plt.suptitle("KDE - Distance to the Meter Fix - ZTL center", fontsize=50)
pdf.savefig()

#boxplots
strClass_delay_pivot.plot(kind='box', subplots=True, figsize=(40,24), layout=(10,10), sharex=False, sharey=False, fontsize=.2)
plt.suptitle("Box & Whisker Plot - Delays - ZTL center", fontsize=50)
pdf.savefig()

strClass_dist_pivot.plot(kind='box', subplots=True, figsize=(40,24), layout=(10,10), sharex=False, sharey=False, fontsize=.2)
plt.suptitle("Box & Whisker Plot - Distance to the Meter Fix - ZTL center", fontsize=50)
pdf.savefig()

plt.close('all')
pdf.close()


TO ADDRESS THE FIRST QUESTION:<br />  

We will assess whether there are statistically significant differences between the delay before vs. after APREQ pre-scheduling. <br />
We will assess overall delay for ORD_DOOGE streamclass and delay for each airport going to ORD_DOOGE.


In [9]:
###############################################################################
#Plots for raw data per streamclass before & after APREQ prescheduling to ORD #
###############################################################################
matplotlib.rcParams['figure.figsize'] = (16.0, 12.0)
matplotlib.style.use('ggplot')
pdf = matplotlib.backends.backend_pdf.PdfPages("ORDDOOGE_Plots_RawData.pdf")

sns.set(style="ticks", color_codes=True)

#Scatter plots
fig, axs = plt.subplots(ncols=2)
ax1 = sns.scatterplot(subset_ORDDOOGE_preAPREQ['meterFixDistance'], subset_ORDDOOGE_preAPREQ['FinalDelay'], edgecolor='none',
                  hue=subset_ORDDOOGE_preAPREQ['FinalDelay'], size=subset_ORDDOOGE_preAPREQ['meterFixDistance'],
                      linewidth=0, alpha = 1, ax=axs[0])
sns.dark_palette("muted purple", input="xkcd")
ax1.set_title('Scatter plot for ORD_DOOGE\n before APREQ prescheduling - raw data')
ax1.set_xlabel('Meter Fix Distance (nm)')
ax1.set_ylabel('Delay (minutes)')
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)

ax2 = sns.scatterplot(subset_ORDDOOGE_pstAPREQ['meterFixDistance'], subset_ORDDOOGE_pstAPREQ['FinalDelay'], edgecolor='none',
                  hue=subset_ORDDOOGE_pstAPREQ['FinalDelay'], size=subset_ORDDOOGE_pstAPREQ['meterFixDistance'],
                      linewidth=0, alpha = 1, ax=axs[1])
sns.dark_palette("palegreen", as_cmap=True)
ax2.set_title('Scatter plot for ORD_DOOGE\n after APREQ prescheduling - raw data')
ax2.set_xlabel('Meter Fix Distance (nm)')
ax2.set_ylabel('Delay (minutes)')
ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)
pdf.savefig()

#Histograms
bins=np.linspace(0, 100, 50)
plt.figure()
plt.hist(subset_ORDDOOGE_preAPREQ['FinalDelay'], bins=bins, alpha=.5, label='before')
plt.hist(subset_ORDDOOGE_pstAPREQ['FinalDelay'], bins=bins, alpha=.5, label='after')
plt.legend(loc='upper right')
plt.suptitle("Histograms - ORD_DOOGE\nDelays before and after APREQ pre-scheduling", fontsize=15)
pdf.savefig()
    
#Density
plt.figure()
sns.distplot(subset_ORDDOOGE_preAPREQ['FinalDelay'], hist = False, kde = True,
                 kde_kws = {'shade': True, 'linewidth': 3}, label='Before')
sns.distplot(subset_ORDDOOGE_pstAPREQ['FinalDelay'], hist = False, kde = True,
                 kde_kws = {'shade': True, 'linewidth': 3}, label='After')
plt.legend(loc='upper right')
plt.suptitle("Density Plot - ORD_DOOGE\nDelays before and after APREQ pre-scheduling", fontsize=15)
pdf.savefig()
    
#Boxplots
plt.figure()

#To prevent multiple boxes being plotted on top of each other:
#1st make tuples
np1=subset_ORDDOOGE_preAPREQ['FinalDelay'].to_numpy()
np_pre=[]
new_col1=np.zeros(np1.size) #pre APREQ-prescheduling data is marked status=0
for x,y in zip(np1, new_col1):
    np_pre.append([x,y])
np2=subset_ORDDOOGE_pstAPREQ['FinalDelay'].to_numpy()
np_pst=[]
new_col2 = np.ones(np2.size) #post APREQ-prescheduling data is marked status=1
for x,y in zip(np2, new_col2):
    np_pst.append([x,y])
np3=np.concatenate((np_pre, np_pst), axis=0) #concat 2 sets of data into one array
    
#then turn tuples into a list
list1=[[float(pair[0]) for pair in np3], [float(pair[1]) for pair in np3]]
    
#then turn the list into a dataframe
subset_ORDDOOGE_byAPREQdate=pd.DataFrame(list(zip(list1[0], list1[1])),
                            columns=['Delay', 'Status'])
subset_ORDDOOGE_byAPREQdate.loc[subset_ORDDOOGE_byAPREQdate['Status'] == 0, ['Status']] = 'Before APREQ Pre-scheduling'
subset_ORDDOOGE_byAPREQdate.loc[subset_ORDDOOGE_byAPREQdate['Status'] == 1, ['Status']] = 'After APREQ Pre-scheduling'

#finally use the dataframe to create the boxplot, so that the 2 sets of data are not on top of each other
sns.boxplot(y='Delay', x='Status', data=subset_ORDDOOGE_byAPREQdate, palette="colorblind", hue='Status')
plt.legend(loc='upper right')
plt.suptitle("Boxplot - ORD_DOOGE\nDelays before and after APREQ pre-scheduling", fontsize=15)

pdf.savefig()

plt.close('all')
pdf.close()

In [23]:
##################################################################################
#Remove outliers for streamclass ORD_DOOGE before and after APREQ prescheduling  #
#Then run statistical tests to compare delay distributions between               #
#pre and post APREQ prescheduling                                                #
##################################################################################

matplotlib.rcParams['figure.figsize'] = (16.0, 12.0)
matplotlib.style.use('ggplot')
pdf = matplotlib.backends.backend_pdf.PdfPages("ORDDOOGE_PDF_PP_Plots.pdf")
 
#######################
#identify outliers using Z for preAPREQ set
print("Identified outliers using Z method for ORD_DOOGE delay data before APREQ prescheduling:")
subset_delay_ORDDOOGE_preAPREQ_noO = pd.DataFrame(outliers_Z(subset_ORDDOOGE_preAPREQ['FinalDelay']))

#######################
#identify outliers using Z for pstAPREQ set
print("Identified outliers using Z method for ORD_DOOGE delay data after APREQ prescheduling:")
subset_delay_ORDDOOGE_pstAPREQ_noO = pd.DataFrame(outliers_Z(subset_ORDDOOGE_pstAPREQ['FinalDelay']))

X_train_delay_ORDDOOGE_preAPREQ, X_test_delay_ORDDOOGE_preAPREQ = train_test_split(subset_delay_ORDDOOGE_preAPREQ_noO,test_size=.5)
X_train_delay_ORDDOOGE_preAPREQ = X_train_delay_ORDDOOGE_preAPREQ.astype(int)
X_test_delay_ORDDOOGE_preAPREQ = X_test_delay_ORDDOOGE_preAPREQ.astype(int)

X_train_delay_ORDDOOGE_pstAPREQ, X_test_delay_ORDDOOGE_pstAPREQ = train_test_split(subset_delay_ORDDOOGE_pstAPREQ_noO,test_size=.5)
X_train_delay_ORDDOOGE_pstAPREQ = X_train_delay_ORDDOOGE_pstAPREQ.astype(int)
X_test_delay_ORDDOOGE_pstAPREQ = X_test_delay_ORDDOOGE_pstAPREQ.astype(int)

#For each airport, run KS test against a normal dist on the test dataset using params derived from train dataset
#For an overview of the KS test: 
#https://www.statisticshowto.datasciencecentral.com/kolmogorov-smirnov-test/

X_train_delay_ORDDOOGE_preAPREQ_xformed = xform(X_train_delay_ORDDOOGE_preAPREQ)
X_test_delay_ORDDOOGE_preAPREQ_xformed = xform(X_test_delay_ORDDOOGE_preAPREQ)

s = pd.DataFrame(X_train_delay_ORDDOOGE_preAPREQ_xformed).describe(include='all')
X_train_delay_ORDDOOGE_preAPREQ_fit_params = [s.loc['mean'], s.loc['std']]

#Here, we will evaluate normality using both the test statistic's critical value and p-value
#Critical value is a point beyond which we reject the null hypothesis.
#P-value on the other hand is defined as the probability to the right of respective statistic (Z, T or chi). 
#The benefit of using p-value is that it calculates a probability estimate, 
#we can test at any desired level of significance by comparing this probability directly with the significance level.
print("\n***Test data distributions for normality:***")
statTest, pTest = st.kstest(X_test_delay_ORDDOOGE_preAPREQ_xformed, 'norm', args=(X_train_delay_ORDDOOGE_preAPREQ_fit_params))
sizeX_test = len(X_test_delay_ORDDOOGE_preAPREQ_xformed)
print("Test for normality using KS: normalized delay distribution for ORD_DOOGE before APREQ-prescheduling")
print("D'sub.05 for delay data w/ sample size of", sizeX_test, " = ", 1.36/np.sqrt(sizeX_test))
print("KS test results against normal dist: D = ", statTest, "p = ", pTest)
print("The null hypothesis is rejected if D > D'sub.05, p < alpha=.05\n")

#Try testing against normality using shapiro-wilk; we'll see that it works better for large sample sizes
#It looks like since critical value for W is harder to compute, evaluate p-value only
statTest, pTest = st.shapiro(X_test_delay_ORDDOOGE_preAPREQ_xformed)
print("Test for normality using Shapiro-Wilk: normalized delay distribution for ORD_DOOGE before APREQ-prescheduling")
print("Shapiro-Wilk test results against normal dist: W = ", statTest, "p = ", pTest)
print("The null hypothesis is rejected if p < alpha=.05")

X_train_delay_ORDDOOGE_pstAPREQ_xformed = xform(X_train_delay_ORDDOOGE_pstAPREQ)
X_test_delay_ORDDOOGE_pstAPREQ_xformed = xform(X_test_delay_ORDDOOGE_pstAPREQ)
        
s = pd.DataFrame(X_train_delay_ORDDOOGE_pstAPREQ_xformed).describe(include='all')
X_train_delay_ORDDOOGE_pstAPREQ_fit_params = [s.loc['mean'], s.loc['std']]
   
statTest, pTest = st.kstest(X_test_delay_ORDDOOGE_pstAPREQ_xformed, 'norm', args=(X_train_delay_ORDDOOGE_pstAPREQ_fit_params))
sizeX_test = len(X_test_delay_ORDDOOGE_pstAPREQ_xformed)
print("\nTest for normality: normalized delay distribution for ORD_DOOGE after APREQ-prescheduling")
print ("D'sub.05 for delay data w/ sample size of", sizeX_test, " = ", 1.36/np.sqrt(sizeX_test))
print("KS test results against normal dist: D = ", statTest, "p = ", pTest)
print("The null hypothesis is rejected if D > D'sub.05, p < alpha=.05\n")

statTest, pTest = st.shapiro(X_test_delay_ORDDOOGE_pstAPREQ_xformed)
print("Test for normality using Shapiro-Wilk: normalized delay distribution for ORD_DOOGE before APREQ-prescheduling")
print("Shapiro-Wilk test results against normal dist: W = ", statTest, "p = ", pTest)
print("The null hypothesis is rejected if p < alpha=.05")

#P-P plots and histograms for normality assessment
left = -1.8   #x coordinate for text insert

fig = plt.figure()

ax = fig.add_subplot(221)
prob= st.probplot(X_test_delay_ORDDOOGE_preAPREQ_xformed.flatten(), dist=st.norm, plot=ax)
ax.set_title('PP Plot:ORD_DOOGE delays before APREQ prescheduling against a normal dist')
top = ax.get_ylim()[1] * 0.75
txt = ax.text(left, top, 'Data is normalized using Box-Cox transformation', verticalalignment='top')
txt.set_bbox(dict(facecolor='k', alpha=0.1))

ax = fig.add_subplot(222)
prob = plt.hist(X_test_delay_ORDDOOGE_preAPREQ_xformed)
ax.set_title('Normalized ORD_DOOGE delays before APREQ prescheduling')

ax = fig.add_subplot(223)
prob = st.probplot(X_test_delay_ORDDOOGE_pstAPREQ_xformed.flatten(), dist=st.norm, plot=ax)
ax.set_title('PP Plot:ORD_DOOGE delays after APREQ prescheduling against a normal dist')
top = ax.get_ylim()[1] * 0.75
txt = ax.text(left, top, 'Data is normalized using Box-Cox transformation', verticalalignment='top')
txt.set_bbox(dict(facecolor='k', alpha=0.1))

ax = fig.add_subplot(224)
prob = plt.hist(X_test_delay_ORDDOOGE_pstAPREQ_xformed)
ax.set_title('Normalized ORD_DOOGE delays after APREQ prescheduling')
    
pdf.savefig()

##################################################################################
#Then compare the delay distribution before vs. after APREQ pre-scheduling to
#assess for statistically significant differences between the two.
#If test for normality run above passes, then use the Welch's T-test, which is the student-T test
#accounting for unequal variances. Here's an interesting blog post about the Welch's T-test:
#http://daniellakens.blogspot.com/2015/01/always-use-welchs-t-test-instead-of.html
#If test for normality above does not pass, then use KS test.

#If there is a statistically significant difference, then assess the effect size
#A small p (≤ alpha = .05), reject the null hypothesis.
#A large p (> alpha = .05) accept the null hypothesis

print("\n***Assess if there is a statistically significant difference between before vs. after APREQ-prescheduling delay distribution for ORD_DOOGE:***\n")
#1st try to normalize the dataset for pre & post APREQ prescheduling 
subset_delay_ORDDOOGE_preAPREQ_xformed = xform(subset_delay_ORDDOOGE_preAPREQ_noO)
subset_delay_ORDDOOGE_pstAPREQ_xformed = xform(subset_delay_ORDDOOGE_pstAPREQ_noO)    
    
sizeX_2samp_test = len(subset_delay_ORDDOOGE_preAPREQ_noO)
sizeY_2samp_test = len(subset_delay_ORDDOOGE_pstAPREQ_noO)
    
print("Sample sizes - Before:", sizeX_2samp_test, "; After:", sizeY_2samp_test)
statTest_delay_2samp_ks, pTest_delay_2samp_ks = st.ks_2samp(subset_delay_ORDDOOGE_preAPREQ_noO.iloc[:,0], subset_delay_ORDDOOGE_pstAPREQ_noO.iloc[:,0])
statTest_delay_2samp_norm_ks, pTest_delay_2samp_norm_ks = st.ks_2samp(subset_delay_ORDDOOGE_preAPREQ_xformed.flatten(), subset_delay_ORDDOOGE_pstAPREQ_xformed.flatten())
statTest_delay_2samp_norm_t, pTest_delay_2samp_norm_t = st.ttest_ind(subset_delay_ORDDOOGE_preAPREQ_xformed.flatten(), subset_delay_ORDDOOGE_pstAPREQ_xformed.flatten(), equal_var=False)


print("Use KS test if normalized data distributions do not pass test for normality:")
print ("D'sub.05 for dataset w/ sample sizes of", sizeX_2samp_test, sizeY_2samp_test, " = ", 1.36*np.sqrt((sizeX_2samp_test + sizeY_2samp_test)/(sizeX_2samp_test*sizeY_2samp_test)))
print("2 sample KS test results for raw data w/o outliers : D = ", statTest_delay_2samp_ks, "p = ", pTest_delay_2samp_ks)
print("2 sample KS test results for normalized data : D = ", statTest_delay_2samp_norm_ks, "p = ", pTest_delay_2samp_norm_ks)
print("The null hypothesis is rejected if D > D'sub.05, p < alpha=.05")

print("\nUse Welch's T-test if normalized data distributions pass test for normality:")
print("2 sample Welch's T-test results for normalized data : T = ", statTest_delay_2samp_norm_t, "p = ", pTest_delay_2samp_norm_t)
print("The null hypothesis is rejected if T > 1.96 and p < alpha=.05")
print("If (sum of 2 sample sizes - 2) < 100, critical value of T can be found at:")
print("https://www.itl.nist.gov/div898/handbook/eda/section3/eda3672.htm")

    
#compute the Cohen's D coefficient since sample sizes are different, use pearson if the sample sizes are the same
print("\n***If a significant difference is found (i.e. p < alpha=.05; reject the null hypothesis), then assess the effect size:***")
print("\nIf data distributions pass normality test, compute effect size using Cohen's D Coefficient: Small: d=0.20; Medium: d=0.50; Large: d=0.80")
print(cohend(subset_delay_ORDDOOGE_preAPREQ_noO.iloc[:,0], subset_delay_ORDDOOGE_pstAPREQ_noO.iloc[:,0]))

#compute the Cliff's Delta coefficient if non-parametric distribution
print("\nIf data distributions do not pass normality test, compute effect size uisng Cliff's Delta:")
print(cliffsDelta(subset_delay_ORDDOOGE_preAPREQ_noO.iloc[:,0], subset_delay_ORDDOOGE_pstAPREQ_noO.iloc[:,0]))
print("\n\n")

plt.close('all')
pdf.close()

Identified outliers using Z method for ORD_DOOGE delay data before APREQ prescheduling:
Number of outliers: 18
No-outlier observations for dataset: 1736
Identified outliers using Z method for ORD_DOOGE delay data after APREQ prescheduling:
Number of outliers: 37
No-outlier observations for dataset: 2428

***Test data distributions for normality:***
Test for normality using KS: normalized delay distribution for ORD_DOOGE before APREQ-prescheduling
D'sub.05 for delay data w/ sample size of 868  =  0.04616140786454489
KS test results against normal dist: D =  0.9780736121441449 p =  0.0
The null hypothesis is rejected if D > D'sub.05, p < alpha=.05

Test for normality using Shapiro-Wilk: normalized delay distribution for ORD_DOOGE before APREQ-prescheduling
Shapiro-Wilk test results against normal dist: W =  0.9801141023635864 p =  1.7176799982365765e-09
The null hypothesis is rejected if p < alpha=.05

Test for normality: normalized delay distribution for ORD_DOOGE after APREQ-preschedul

In [11]:
##############################################################################
#Plot delays before & after APREQ prescheduling for airports per streamclass #
##############################################################################

#origApt_delay_ORDDOOGE_preAPREQ.drop = ['ATL','MYR','MCO','AGS','JAX']
#origApt_delay_ORDDOOGE_pstAPREQ.drop = ['BHM','GSO','MYR']
matplotlib.rcParams['figure.figsize'] = (16.0, 12.0)
matplotlib.style.use('ggplot')
plt.rcParams.update({'figure.max_open_warning': 0})

origAptList = [
    'CHS',
    'SAV',
    'CAE',
    'AVL',
    'GSP',
    'CLT',
    'RDU']

pdf = matplotlib.backends.backend_pdf.PdfPages("ORDDOOGE_Plots_byAirports.pdf")

bins=np.linspace(0, 100, 50)

for aptName in origAptList:
    subset_byAirport_ORDDOOGE_preAPREQ = subset_ORDDOOGE_preAPREQ[subset_ORDDOOGE_preAPREQ['origApt']==aptName]
    subset_byAirport_ORDDOOGE_pstAPREQ = subset_ORDDOOGE_pstAPREQ[subset_ORDDOOGE_pstAPREQ['origApt']==aptName]

    #Histograms
    plt.figure()
    plt.hist(subset_byAirport_ORDDOOGE_preAPREQ['FinalDelay'], bins=bins, alpha=.5, label='before')
    plt.hist(subset_byAirport_ORDDOOGE_pstAPREQ['FinalDelay'], bins=bins, alpha=.5, label='after')
    plt.legend(loc='upper right')
    plt.suptitle("Histograms - " + aptName + " to ORD_DOOGE\nDelays before and after APREQ pre-scheduling", fontsize=15)
    pdf.savefig()
    
    #Density
    plt.figure()
    sns.distplot(subset_byAirport_ORDDOOGE_preAPREQ['FinalDelay'], hist = False, kde = True,
                 kde_kws = {'shade': True, 'linewidth': 3}, label='Before')
    sns.distplot(subset_byAirport_ORDDOOGE_pstAPREQ['FinalDelay'], hist = False, kde = True,
                 kde_kws = {'shade': True, 'linewidth': 3}, label='After')
    plt.legend(loc='upper right')
    plt.suptitle("Density Plot - " + aptName + " to ORD_DOOGE\nDelays before and after APREQ pre-scheduling", fontsize=15)
    pdf.savefig()
    
    #Boxplot
    plt.figure()
    
    #to prevent having the boxes being plotted on top of each other:
    #1st make an array of tuples
    np1=subset_byAirport_ORDDOOGE_preAPREQ['FinalDelay'].to_numpy()
    np_pre=[]
    new_col1=np.zeros(np1.size) #pre APREQ-prescheduling data is marked status=0
    for x,y in zip(np1, new_col1):
        np_pre.append([x,y])
    np2=subset_byAirport_ORDDOOGE_pstAPREQ['FinalDelay'].to_numpy()
    np_pst=[]
    new_col2 = np.ones(np2.size) #post APREQ-prescheduling data is marked status=1
    for x,y in zip(np2, new_col2):
        np_pst.append([x,y])
    np3=np.concatenate((np_pre, np_pst), axis=0) #concat 2 sets of data into one array
    
    #then turn the array of tuples into a list
    list1=[[float(pair[0]) for pair in np3], [float(pair[1]) for pair in np3]]
    
    #then turn the list into a data frame
    subset_DelayByAirport_ORDDOOGE_byAPREQdate=pd.DataFrame(list(zip(list1[0], list1[1])),
                            columns=['Delay', 'Status'])
    subset_DelayByAirport_ORDDOOGE_byAPREQdate.loc[subset_DelayByAirport_ORDDOOGE_byAPREQdate['Status'] == 0, ['Status']] = 'Before APREQ Pre-scheduling'
    subset_DelayByAirport_ORDDOOGE_byAPREQdate.loc[subset_DelayByAirport_ORDDOOGE_byAPREQdate['Status'] == 1, ['Status']] = 'After APREQ Pre-scheduling'

    #finally use the dataframe to create the boxplot, so that the 2 sets of data are not on top of each other
    sns.boxplot(y='Delay', x='Status', data=subset_DelayByAirport_ORDDOOGE_byAPREQdate, palette="colorblind", hue='Status')
    plt.legend(loc='upper right')
    plt.suptitle("Boxplot - " + aptName + " to ORD_DOOGE\nDelays before and after APREQ pre-scheduling", fontsize=15)
    pdf.savefig()

plt.close('all')
pdf.close()
                                                      

In [24]:
###############################################################
#Remove outliers for airports to streamclass ORD_DOOGE        #
#then find best fit pdf for ORDDOOGE delay by airport dataset #
#before and after APREQ prescheduling                         #
#Also run statistical tests to compare delay distributions    #
#between pre and post APREQ prescheduling                     #
###############################################################
matplotlib.rcParams['figure.figsize'] = (16.0, 12.0)
matplotlib.style.use('ggplot')
pdf = matplotlib.backends.backend_pdf.PdfPages("ORDDOOGE_PDF_PP_plots_byAirports.pdf")

origAptList = [
    'CHS',
    'SAV',
    'CAE',
    'AVL',
    'GSP',
    'CLT',
    'RDU']

for aptName in origAptList:
    matplotlib.rcParams['figure.figsize'] = (16.0, 12.0)
    matplotlib.style.use('ggplot')
    
    subset_byAirport_ORDDOOGE_preAPREQ = subset_ORDDOOGE_preAPREQ[subset_ORDDOOGE_preAPREQ['origApt']==aptName]
    subset_byAirport_ORDDOOGE_pstAPREQ = subset_ORDDOOGE_pstAPREQ[subset_ORDDOOGE_pstAPREQ['origApt']==aptName]
    print('Airport: ', aptName)
    
    #######################
    #identify outliers using Z for preAPREQ set
    print("Identified outliers using Z method for delay data for:", aptName)
    subset_DelayByAirport_ORDDOOGE_preAPREQ_noO = pd.DataFrame(outliers_Z(subset_byAirport_ORDDOOGE_preAPREQ['FinalDelay']))

    #######################
    #identify outliers using Z for pstAPREQ set
    print("Identified outliers using Z method for delay data for:", aptName)
    subset_DelayByAirport_ORDDOOGE_pstAPREQ_noO = pd.DataFrame(outliers_Z(subset_byAirport_ORDDOOGE_pstAPREQ['FinalDelay']))

    print("\n")
    #######################
    #Perform evaluation of best fit distribution for pre-APREQ by
    #iterating through a list of dist and evaluate fit by computing the sum squared error
    X_train_DelayByAirport_ORDDOOGE_preAPREQ, X_test_DelayByAirport_ORDDOOGE_preAPREQ = train_test_split(subset_DelayByAirport_ORDDOOGE_preAPREQ_noO,test_size=.5)
    X_train_DelayByAirport_ORDDOOGE_preAPREQ = X_train_DelayByAirport_ORDDOOGE_preAPREQ.astype(int)
    X_test_DelayByAirport_ORDDOOGE_preAPREQ = X_test_DelayByAirport_ORDDOOGE_preAPREQ.astype(int)

    X_train_DelayByAirport_ORDDOOGE_pstAPREQ, X_test_DelayByAirport_ORDDOOGE_pstAPREQ = train_test_split(subset_DelayByAirport_ORDDOOGE_pstAPREQ_noO,test_size=.5)
    X_train_DelayByAirport_ORDDOOGE_pstAPREQ = X_train_DelayByAirport_ORDDOOGE_pstAPREQ.astype(int)
    X_test_DelayByAirport_ORDDOOGE_pstAPREQ = X_test_DelayByAirport_ORDDOOGE_pstAPREQ.astype(int)

    ###################################
    # Create models from data
    ###################################

    print("***Find best fit pdf for ORDDOOGE delay by airport dataset***")
    ############################################################
    # Find best fit pdf for ORDDOOGE delay dataset

    # use half of dataset (randomly drawn) to find distribution params utilizing fit function
    #data = subset_Delay_ORDDOOGE_noO #if use full dataset
    
    #Pre APREQ Prescheduling
    data_DelayByAirport_preAPREQ = X_train_DelayByAirport_ORDDOOGE_preAPREQ
    
    # Plot for comparison
    plt.figure()
    ax1 = data_DelayByAirport_preAPREQ.plot(kind='hist', bins=50, density=True, alpha=0.5, legend=False)

    # Save plot limits
    dataYLim1 = ax1.get_ylim()

    # Find best fit distribution
    best_fit_DelayByAirport_preAPREQ_name, best_fit_DelayByAirport_preAPREQ_params = best_fit_distribution(data_DelayByAirport_preAPREQ, 200, ax1)
    best_DelayByAirport_preAPREQ_dist = getattr(st, best_fit_DelayByAirport_preAPREQ_name)

    # Update plots
    ax1.set_ylim(dataYLim1)
    ax1.set_title(u'Delays at ' + aptName + ' to ORD_DOOGE before APREQ Prescheduling\n Selected Fitted Distributions')
    ax1.set_xlabel('Minutes')
    ax1.set_ylabel('Frequency')
    pdf.savefig()

    #Post APREQ Prescheduling
    data_DelayByAirport_pstAPREQ = X_train_DelayByAirport_ORDDOOGE_pstAPREQ
    plt.figure()
    ax2 = data_DelayByAirport_pstAPREQ.plot(kind='hist', bins=50, density=True, alpha=0.5, legend=False)
    dataYLim2 = ax2.get_ylim()
    best_fit_DelayByAirport_pstAPREQ_name, best_fit_DelayByAirport_pstAPREQ_params = best_fit_distribution(data_DelayByAirport_pstAPREQ, 200, ax2)
    best_DelayByAirport_pstAPREQ_dist = getattr(st, best_fit_DelayByAirport_pstAPREQ_name)
    ax2.set_ylim(dataYLim2)
    ax2.set_title(u'Delays at ' + aptName + ' to ORD_DOOGE after APREQ Prescheduling\n Selected Fitted Distributions')
    ax2.set_xlabel('Minutes')
    ax2.set_ylabel('Frequency')    
    pdf.savefig()
    
    #######################    
    # Make PDF with best params 
    pdf_DelayByAirport_preAPREQ = make_pdf(best_DelayByAirport_preAPREQ_dist, best_fit_DelayByAirport_preAPREQ_params)
    pdf_DelayByAirport_pstAPREQ = make_pdf(best_DelayByAirport_pstAPREQ_dist, best_fit_DelayByAirport_pstAPREQ_params)

    ##############################################    
    # Display
    
    #pre-APREQ prescheduling 
    plt.figure()
    ax = pdf_DelayByAirport_preAPREQ.plot(lw=2, label='pdf', legend=True)
    data_DelayByAirport_preAPREQ.plot(kind='hist', bins=50, density=True, label='Data', legend=False, alpha=0.5, ax=ax)

    param_DelayByAirport_preAPREQ_names = (best_DelayByAirport_preAPREQ_dist.shapes + ', loc, scale').split(', ') if best_DelayByAirport_preAPREQ_dist.shapes else ['loc', 'scale']
    param_DelayByAirport_preAPREQ_str = ', '.join(['{}={:0.2f}'.format(k,v) for k,v in zip(param_DelayByAirport_preAPREQ_names, best_fit_DelayByAirport_preAPREQ_params)])
    dist_DelayByAirport_preAPREQ_str = '{}({})'.format(best_fit_DelayByAirport_preAPREQ_name, param_DelayByAirport_preAPREQ_str)

    ax.set_title(u'Delays at ' + aptName + ' to ORD_DOOGE before APREQ prescheduling with best fit distribution \n' + dist_DelayByAirport_preAPREQ_str)
    ax.set_xlabel('Minutes')
    ax.set_ylabel('Frequency')   
    pdf.savefig()

    #post APREQ prescheduling
    plt.figure()
    ax = pdf_DelayByAirport_pstAPREQ.plot(lw=2, label='pdf', legend=True)
    data_DelayByAirport_pstAPREQ.plot(kind='hist', bins=50, density=True, label='Data', legend=False, alpha=0.5, ax=ax)

    param_DelayByAirport_pstAPREQ_names = (best_DelayByAirport_pstAPREQ_dist.shapes + ', loc, scale').split(', ') if best_DelayByAirport_pstAPREQ_dist.shapes else ['loc', 'scale']
    param_DelayByAirport_pstAPREQ_str = ', '.join(['{}={:0.2f}'.format(k,v) for k,v in zip(param_DelayByAirport_pstAPREQ_names, best_fit_DelayByAirport_pstAPREQ_params)])
    dist_DelayByAirport_pstAPREQ_str = '{}({})'.format(best_fit_DelayByAirport_pstAPREQ_name, param_DelayByAirport_pstAPREQ_str)

    ax.set_title(u'Delays at ' + aptName + ' to ORD_DOOGE after APREQ Prescheduling with best fit distribution \n' + dist_DelayByAirport_pstAPREQ_str)
    ax.set_xlabel('Minutes')
    ax.set_ylabel('Frequency')
    pdf.savefig()

    #################################################################################################
    #Run statistical tests to compare delay distributions between pre and post APREQ prescheduling  #
    #################################################################################################
    
    #For each airport, run KS test against a normal dist on the test dataset using params derived from train dataset
    X_train_DelayByAirport_ORDDOOGE_preAPREQ_xformed = xform(X_train_DelayByAirport_ORDDOOGE_preAPREQ)
    X_test_DelayByAirport_ORDDOOGE_preAPREQ_xformed = xform(X_test_DelayByAirport_ORDDOOGE_preAPREQ)

    s = pd.DataFrame(X_train_DelayByAirport_ORDDOOGE_preAPREQ_xformed).describe(include='all')
    X_train_DelayByAirport_ORDDOOGE_preAPREQ_fit_params = [s.loc['mean'], s.loc['std']]

    print("\n***Test data distributions for normality:***")
    statTest, pTest = st.kstest(X_test_DelayByAirport_ORDDOOGE_preAPREQ_xformed, 'norm', args=(X_train_DelayByAirport_ORDDOOGE_preAPREQ_fit_params))
    sizeX_test = len(X_test_DelayByAirport_ORDDOOGE_preAPREQ_xformed)
    print("Test for normality using KS: normalized delay distribution for", aptName, "before APREQ-prescheduling")
    print("D'sub.05 for ", aptName, " delay dataset w/ sample size of", sizeX_test, " = ", 1.36/np.sqrt(sizeX_test))
    print("Find D'sub.05 critical value from the table below if sample size < 35")
    print("KS test results against normal dist: D = ", statTest, "p = ", pTest)
    print("The null hypothesis is rejected if D > D'sub.05, p < alpha=.05\n")

    #Try testing against normality using shapiro-wilk; we'll see that it works better for large sample sizes
    #It looks like since critical value for W is harder to compute, evaluate p-value only
    statTest, pTest = st.shapiro(X_test_DelayByAirport_ORDDOOGE_preAPREQ_xformed)
    print("Test for normality using Shapiro-Wilk: normalized delay distribution for ORD_DOOGE before APREQ-prescheduling")
    print("Shapiro-Wilk test results against normal dist: W = ", statTest, "p = ", pTest)
    print("The null hypothesis is rejected if p < alpha=.05")

    X_train_DelayByAirport_ORDDOOGE_pstAPREQ_xformed = xform(X_train_DelayByAirport_ORDDOOGE_pstAPREQ)
    X_test_DelayByAirport_ORDDOOGE_pstAPREQ_xformed = xform(X_test_DelayByAirport_ORDDOOGE_pstAPREQ)
        
    s = pd.DataFrame(X_train_DelayByAirport_ORDDOOGE_pstAPREQ_xformed).describe(include='all')
    X_train_DelayByAirport_ORDDOOGE_pstAPREQ_fit_params = [s.loc['mean'], s.loc['std']]
   
    statTest, pTest = st.kstest(X_test_DelayByAirport_ORDDOOGE_pstAPREQ_xformed, 'norm', args=(X_train_DelayByAirport_ORDDOOGE_pstAPREQ_fit_params))
    sizeX_test = len(X_test_DelayByAirport_ORDDOOGE_pstAPREQ_xformed)
    print("\nTest for normality using KS: normalized delay distribution for", aptName, "after APREQ-prescheduling")
    print ("D'sub.05 for ", aptName, " delay dataset w/ sample size of", sizeX_test, " = ", 1.36/np.sqrt(sizeX_test))
    print("Find D'sub.05 critical value from the table below if sample size < 35")
    print("KS test results against normal dist: D = ", statTest, "p = ", pTest)
    print("The null hypothesis is rejected if D > D'sub.05, p < alpha=.05\n")

    #Try testing against normality using shapiro-wilk; we'll see that it works better for large sample sizes
    #It looks like since critical value for W is harder to compute, evaluate p-value only
    statTest, pTest = st.shapiro(X_test_DelayByAirport_ORDDOOGE_pstAPREQ_xformed)
    print("Test for normality using Shapiro-Wilk: normalized delay distribution for ORD_DOOGE before APREQ-prescheduling")
    print("Shapiro-Wilk test results against normal dist: W = ", statTest, "p = ", pTest)
    print("The null hypothesis is rejected if p < alpha=.05")    
    
    #P-P plots and histograms for normality assessment
    left = -1.8   #x coordinate for text insert

    fig = plt.figure()

    ax = fig.add_subplot(221)
    prob= st.probplot(X_test_DelayByAirport_ORDDOOGE_preAPREQ_xformed.flatten(), dist=st.norm, plot=ax)
    ax.set_title('PP Plot: Delays data before APREQ presscheduling against a normal dist for ' + aptName)
    top = ax.get_ylim()[1] * 0.75
    txt = ax.text(left, top, 'Data is normalized using Box-Cox transformation', verticalalignment='top')
    txt.set_bbox(dict(facecolor='k', alpha=0.1))

    ax = fig.add_subplot(222)
    prob = plt.hist(X_test_DelayByAirport_ORDDOOGE_preAPREQ_xformed)
    ax.set_title('Histogram of normalized delays data for ' + aptName)

    ax = fig.add_subplot(223)
    prob = st.probplot(X_test_DelayByAirport_ORDDOOGE_pstAPREQ_xformed.flatten(), dist=st.norm, plot=ax)
    ax.set_title('PP Plot: Delays data after APREQ presscheduling against a normal dist for ' + aptName)
    top = ax.get_ylim()[1] * 0.75
    txt = ax.text(left, top, 'Data is normalized using Box-Cox transformation', verticalalignment='top')
    txt.set_bbox(dict(facecolor='k', alpha=0.1))

    ax = fig.add_subplot(224)
    prob = plt.hist(X_test_DelayByAirport_ORDDOOGE_pstAPREQ_xformed)
    ax.set_title('Histogram of normalized delays data for ' + aptName)    
    pdf.savefig()    
      
    ##################################################################################
    #Then compare the delay distribution before vs. after APREQ pre-scheduling to
    #assess for statistically significant differences between the two.
    #If there is a statistically significant difference, then assess the effect size
    #A small p (≤ alpha = .05), reject the null hypothesis.
    #A large p (> alpha = .05) accept the null hypothesis

    print("\n***Assess if there is a statistically significant difference between before vs. after APREQ-prescheduling delay distribution for airport ", aptName, "***\n")

    #1st try to normalize the dataset for pre & post APREQ prescheduling 
    subset_DelayByAirport_ORDDOOGE_preAPREQ_xformed = xform(subset_DelayByAirport_ORDDOOGE_preAPREQ_noO)
    subset_DelayByAirport_ORDDOOGE_pstAPREQ_xformed = xform(subset_DelayByAirport_ORDDOOGE_pstAPREQ_noO)    
    
    sizeX_2samp_test = len(subset_DelayByAirport_ORDDOOGE_preAPREQ_noO)
    sizeY_2samp_test = len(subset_DelayByAirport_ORDDOOGE_pstAPREQ_noO)
    
    print("Sample sizes - Before:", sizeX_2samp_test, "; After:", sizeY_2samp_test)
    statTest_delay_2samp_ks, pTest_delay_2samp_ks = st.ks_2samp(subset_DelayByAirport_ORDDOOGE_preAPREQ_noO.iloc[:,0], subset_DelayByAirport_ORDDOOGE_pstAPREQ_noO.iloc[:,0])
    statTest_delay_2samp_norm_ks, pTest_delay_2samp_norm_ks = st.ks_2samp(subset_DelayByAirport_ORDDOOGE_preAPREQ_xformed.flatten(), subset_DelayByAirport_ORDDOOGE_pstAPREQ_xformed.flatten())
    statTest_delay_2samp_norm_t, pTest_delay_2samp_norm_t = st.ttest_ind(subset_DelayByAirport_ORDDOOGE_preAPREQ_xformed.flatten(), subset_DelayByAirport_ORDDOOGE_pstAPREQ_xformed.flatten(), equal_var=False)


    print("Use KS test if normalized data distributions do not pass test for normality:")
    print ("D'sub.05 for dataset w/ sample sizes of", sizeX_2samp_test, sizeY_2samp_test, " = ", 1.36*np.sqrt((sizeX_2samp_test + sizeY_2samp_test)/(sizeX_2samp_test*sizeY_2samp_test)))
    print("2 sample KS test results for raw data w/o outliers : D = ", statTest_delay_2samp_ks, "p = ", pTest_delay_2samp_ks)
    print("2 sample KS test results for normalized data : D = ", statTest_delay_2samp_norm_ks, "p = ", pTest_delay_2samp_norm_ks)
    print("The null hypothesis is rejected if D > D'sub.05, p < alpha=.05")

    print("\nUse Welch's T-test if normalized data distributions pass test for normality:")
    print("2 sample Welch's T-test results for normalized data : T = ", statTest_delay_2samp_norm_t, "p = ", pTest_delay_2samp_norm_t)
    print("The null hypothesis is rejected if T > 1.96, p < alpha=.05")
    print("If (sum of 2 sample sizes - 2) < 100, critical value of T can be found at:")
    print("https://www.itl.nist.gov/div898/handbook/eda/section3/eda3672.htm")
    
    #compute the Cohen's D coefficient since sample sizes are different, can use Pearson if the sample sizes are the same
    print("\n***If a significant difference is found (i.e. p < alpha=.05; reject the null hypothesis), then assess the effect size:***")
    print("Compute effect size using Cohen's D Coefficient: Small: d=0.20; Medium: d=0.50; Large: d=0.80")
    print(cohend(subset_DelayByAirport_ORDDOOGE_preAPREQ_noO.iloc[:,0], subset_DelayByAirport_ORDDOOGE_pstAPREQ_noO.iloc[:,0]))

    #compute the Cliff's Delta coefficient if non-parametric distribution
    print("\nIf data distributions do not pass normality test, compute effect size using Cliff's Delta:")
    print(cliffsDelta(subset_DelayByAirport_ORDDOOGE_preAPREQ_noO.iloc[:,0], subset_DelayByAirport_ORDDOOGE_pstAPREQ_noO.iloc[:,0]))

    print("\n\n")
    plt.close('all')
    
    
pdf.close()

Airport:  CHS
Identified outliers using Z method for delay data for: CHS
Number of outliers: 0
No-outlier observations for dataset: 14
Identified outliers using Z method for delay data for: CHS
Number of outliers: 0
No-outlier observations for dataset: 43


***Find best fit pdf for ORDDOOGE delay by airport dataset***
dweibull  distribution parameters:  (1, 0, 8.000011643863989)
SSE for  dweibull :  11.88669162823157
expon  distribution parameters:  (1.0, 7.0)
SSE for  expon :  11.536028961302767
gamma  distribution parameters:  (0.7892448488688286, 0, 10.13627141370116)
SSE for  gamma :  11.565530482394141
invweibull  distribution parameters:  (1, 0, 2.1440429687500027)
SSE for  invweibull :  11.060943083696516
lognorm  distribution parameters:  (1.2335757913352383, 0.0, 3.766405163294681)
SSE for  lognorm :  11.313834634479413
norm  distribution parameters:  (8.0, 9.54687682663064)
SSE for  norm :  12.063631241494926
weibull_min  distribution parameters:  (1, 0, 7.99999609471802)
SSE

dweibull  distribution parameters:  (1, 0, 9.466650546494492)
SSE for  dweibull :  3.203876281303342
expon  distribution parameters:  (1.0, 8.466666666666667)
SSE for  expon :  3.080526737646565
gamma  distribution parameters:  (1.6628321571033637, 0, 5.693098143565795)
SSE for  gamma :  3.112531658299777
invweibull  distribution parameters:  (1, 0, 4.485546875000008)
SSE for  invweibull :  3.1089584041885523
lognorm  distribution parameters:  (0.886048254780868, 0.0, 6.806613303370268)
SSE for  lognorm :  3.112267434942389
norm  distribution parameters:  (9.466666666666667, 6.829999186595039)
SSE for  norm :  3.1883975901154713
weibull_min  distribution parameters:  (1, 0, 9.46669921875002)
SSE for  weibull_min :  3.083188054270517
weibull_max  distribution parameters:  (1, 0, 5.329070518200764e-16)
SSE for  weibull_max :  3.4823959762231222

***Test data distributions for normality:***
Test for normality using KS: normalized delay distribution for CAE before APREQ-prescheduling
D'sub

dweibull  distribution parameters:  (1, 0, 14.569328663909591)
SSE for  dweibull :  0.6068936778674443
expon  distribution parameters:  (1.0, 13.569306930693068)
SSE for  expon :  0.5681273075560853
gamma  distribution parameters:  (1.5798252891665236, 0, 9.222100083218361)
SSE for  gamma :  0.571132022838132
invweibull  distribution parameters:  (1, 0, 6.339648437500011)
SSE for  invweibull :  0.5870016492719763
lognorm  distribution parameters:  (0.9239658364444332, 0.0, 10.27986268399382)
SSE for  lognorm :  0.5747430488070814
norm  distribution parameters:  (14.569306930693068, 10.854467005574271)
SSE for  norm :  0.5984335325587998
weibull_min  distribution parameters:  (1, 0, 14.569335937500028)
SSE for  weibull_min :  0.5671082775475977
weibull_max  distribution parameters:  (1, 0, 5.329070518200764e-16)
SSE for  weibull_max :  0.7115948638416962
dweibull  distribution parameters:  (1, 0, 16.618654208917533)
SSE for  dweibull :  0.4113265082660056
expon  distribution parameters:

dweibull  distribution parameters:  (1, 0, 13.36541037992686)
SSE for  dweibull :  0.7099537352600158
expon  distribution parameters:  (1.0, 12.365384615384615)
SSE for  expon :  0.6664823534847034
gamma  distribution parameters:  (1.7238700591900789, 0, 7.753127646792611)
SSE for  gamma :  0.6656016072547731
invweibull  distribution parameters:  (1, 0, 6.219335937500011)
SSE for  invweibull :  0.6886644718309717
lognorm  distribution parameters:  (0.8794341931019228, 0.0, 9.731894757959441)
SSE for  lognorm :  0.672288962851304
norm  distribution parameters:  (13.365384615384615, 9.643957546555425)
SSE for  norm :  0.6898117424259818
weibull_min  distribution parameters:  (1, 0, 13.365429687500026)
SSE for  weibull_min :  0.6643920608742916
weibull_max  distribution parameters:  (1, 0, 5.329070518200764e-16)
SSE for  weibull_max :  0.8303004822693238

***Test data distributions for normality:***
Test for normality using KS: normalized delay distribution for RDU before APREQ-preschedul

<img src="KS_Critical_Values.png" alt="Drawing" style="width: 500px;"/>

SOME ADDITIONAL OBSERVATIONS:<br />
We notice that the number of APREQ flights after APREQ Prescheduling was much higher than the number of APREQ flights before APREQ prescheduling. This is possibly due to seasonal timing (stormy season perhaps?). However, it probably skews our assessment of the differences between the 2 datasets. A more accurate comparison may be done for 2 datasets at the same time of different years.<br />

----------------------------------------------------------------------------------------------------------------<br />

ONTO THE SECOND QUESTION:<br />  
We will assess whether there is a linear relationship between overall delay assigned by TBFM and the flight's distance to the meter fix. First, we will perform a quick test to see if there is potential for a linear relationship by running the linear regression model on normalized data.  If a linear relationship is found, we will perform more rigorous tesing of assumptions as listed here.<br />

The assumptions and conditions we will evaluate any time we generate a simple or multiple regression analysis will encompass:<br />
• Linearity<br />
• Homoscedasticity (Constant Variance)<br />
• Normality<br />
• Independence of Error<br />
• Outlier and Influential Observation Analysis<br />

In [13]:
############################################
#Remove outliers for data per streamclass  #
############################################

subset_Delay_ORDDOOGE_noO = pd.DataFrame(outliers_IQR(subset_ORDDOOGE['FinalDelay']))
#subset_Delay_ORDDOOGE_noO = subset_ORDDOOGE['FinalDelay'] #if you don't want to remove outliers

print("\n")

subset_meterFixDistance_ORDDOOGE_noO = pd.DataFrame(outliers_IQR(subset_ORDDOOGE['meterFixDistance']))
#subset_meterFixDistance_ORDDOOGE_noO = subset_ORDDOOGE['meterFixDistance'] #if you don't want to remove outliers


Percentiles for dataset: 25th=5.000, 75th=18.000, IQR=13.000
Number of outliers: 154
No-outlier observations for dataset: 4279


Percentiles for dataset: 25th=136.039, 75th=211.839, IQR=75.800
Number of outliers: 56
No-outlier observations for dataset: 4377


In [14]:
######################################################################################
#Find best fit pdf for ORDDOOGE delay and meterFixDistance dataset by SSE comparison #
#We only use half of the dataset to find the dist params, leave the other half for   #
#goodness-of-fit statistical tests                                                   #
######################################################################################

#Perform evaluation of best fit distribution by
#iterating through a list of dist and evaluate fit by computing the sum of squared error
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (16.0, 12.0)
matplotlib.style.use('ggplot')
pdf = matplotlib.backends.backend_pdf.PdfPages("ORDDOOGE_PDF_plots.pdf")

X_train_Delay_ORDDOOGE, X_test_Delay_ORDDOOGE = train_test_split(subset_Delay_ORDDOOGE_noO,test_size=.5)
X_train_Delay_ORDDOOGE = X_train_Delay_ORDDOOGE.astype(int)
X_test_Delay_ORDDOOGE = X_test_Delay_ORDDOOGE.astype(int)

X_train_meterFixDistance_ORDDOOGE, X_test_meterFixDistance_ORDDOOGE = train_test_split(subset_meterFixDistance_ORDDOOGE_noO,test_size=.5)
X_train_meterFixDistance_ORDDOOGE = X_train_meterFixDistance_ORDDOOGE.astype(float)
X_test_meterFixDistance_ORDDOOGE = X_test_meterFixDistance_ORDDOOGE.astype(float)

print("Find best fit pdf for ORDDOOGE delay dataset")
############################################################
# Find best fit pdf for ORDDOOGE delay dataset

# use half of dataset (randomly drawn) to find distribution params utilizing fit function
#data = subset_Delay_ORDDOOGE_noO
data_Delay = X_train_Delay_ORDDOOGE

# Plot for comparison
plt.figure()
ax = data_Delay.plot(kind='hist', bins=50, density=True, alpha=0.5, legend=False)
    
# Save plot limits
dataYLim = ax.get_ylim()

# Find best fit distribution
best_fit_Delay_name, best_fit_Delay_params = best_fit_distribution(data_Delay, 200, ax)
best_Delay_dist = getattr(st, best_fit_Delay_name)

# Update plots
ax.set_ylim(dataYLim)
ax.set_title(u'Delays to ORD_DOOGE\n Selected Fitted Distributions')
ax.set_xlabel('Minutes')
ax.set_ylabel('Frequency')

pdf.savefig()

# Make PDF with best params 
pdf_Delay = make_pdf(best_Delay_dist, best_fit_Delay_params)

# Display
plt.figure()
ax = pdf_Delay.plot(lw=2, label='pdf', legend=True)
data_Delay.plot(kind='hist', bins=50, density=True, alpha=0.5, label='Data', legend=False, ax=ax)

param_Delay_names = (best_Delay_dist.shapes + ', loc, scale').split(', ') if best_Delay_dist.shapes else ['loc', 'scale']
param_Delay_str = ', '.join(['{}={:0.2f}'.format(k,v) for k,v in zip(param_Delay_names, best_fit_Delay_params)])
dist_Delay_str = '{}({})'.format(best_fit_Delay_name, param_Delay_str)

ax.set_title(u'Delays to ORD_DOOGE with best fit distribution \n' + dist_Delay_str)
ax.set_xlabel('Minutes')
ax.set_ylabel('Frequency')

pdf.savefig()

print("\nFind best fit pdf for ORDDOOGE meterFixDistance dataset")
############################################################
# Find best fit pdf for ORDDOOGE meterFixDistance dataset

# use half of dataset (randomly drawn) to find distribution params utilizing fit function
data_mfd = X_train_meterFixDistance_ORDDOOGE

# Plot for comparison
plt.figure()
ax = data_mfd.plot(kind='hist', bins=50, density=True, alpha=0.5, legend=False)

# Save plot limits
dataYLim = ax.get_ylim()

# Find best fit distribution
best_fit_mfd_name, best_fit_mfd_params = best_fit_distribution(data_mfd, 200, ax)
best_mfd_dist = getattr(st, best_fit_mfd_name)

# Update plots
ax.set_ylim(dataYLim)
ax.set_title(u'Distance to the meter fix - ORD_DOOGE \n Selected Fitted Distributions')
ax.set_xlabel('nm')
ax.set_ylabel('Frequency')

pdf.savefig()

# Make PDF with best params 
pdf_mfd = make_pdf(best_mfd_dist, best_fit_mfd_params)

# Display
plt.figure()
ax = pdf_mfd.plot(lw=2, label='pdf', legend=True)
data_mfd.plot(kind='hist', bins=50, density=True, alpha=0.5, label='Data', legend=False, ax=ax)

param_mfd_names = (best_mfd_dist.shapes + ', loc, scale').split(', ') if best_mfd_dist.shapes else ['loc', 'scale']
param_mfd_str = ', '.join(['{}={:0.2f}'.format(k,v) for k,v in zip(param_mfd_names, best_fit_mfd_params)])
dist_mfd_str = '{}({})'.format(best_fit_mfd_name, param_mfd_str)

ax.set_title(u'Distance to the meter fix ORD_DOOGE with best fit distribution \n' + dist_mfd_str)
ax.set_xlabel('nm')
ax.set_ylabel('Frequency')

pdf.savefig()

plt.close('all')
pdf.close()


Find best fit pdf for ORDDOOGE delay dataset
dweibull  distribution parameters:  (1, 0, 12.080877634438345)
SSE for  dweibull :  1.0813190358935625
expon  distribution parameters:  (1.0, 11.08087891538102)
SSE for  expon :  1.0152343115729416
gamma  distribution parameters:  (1.5777643933447205, 0, 7.656960041905007)
SSE for  gamma :  1.0234101086367997
invweibull  distribution parameters:  (1, 0, 5.358886718750009)
SSE for  invweibull :  1.0481759308586276
lognorm  distribution parameters:  (0.9161486488590146, 0.0, 8.519850127757808)
SSE for  lognorm :  1.0301266905766857
norm  distribution parameters:  (12.08087891538102, 9.18377657590736)
SSE for  norm :  1.069083296055466
weibull_min  distribution parameters:  (1, 0, 12.080859375000024)
SSE for  weibull_min :  1.014999147823064
weibull_max  distribution parameters:  (1, 0, 5.329070518200764e-16)
SSE for  weibull_max :  1.2448091040099487

Find best fit pdf for ORDDOOGE meterFixDistance dataset
dweibull  distribution parameters:  (

In [15]:
#########################################################################
#Use maximum likelihood estimates (MLE) to find best fit pdf parameters #
#########################################################################

##########
# THIS SECTION NEEDS WORK
##########

#intro to MLE: https://machinelearningmastery.com/what-is-maximum-likelihood-estimation-in-machine-learning/
#also: https://towardsdatascience.com/maximum-likelihood-estimation-explained-normal-distribution-6207b322e47f

#The best distribution for your data can be determined by several different ways:
#1: the one that gives you the highest log likelihood (aka LL), or lowest negative LL
#2: the one that gives you the smallest AIC, BIC or BICc values 
#(see wiki: http://en.wikipedia.org/wiki/Akaike_information_criterion, 
# basically can be viewed as log likelihood adjusted for number of parameters, as distribution with more parameters are expected to fit better)
#3: the one that maximize the Bayesian posterior probability.
#(see wiki: http://en.wikipedia.org/wiki/Posterior_probability)

#Of course, if you already have a distribution that should describe your data (based on the theories in your particular field)
#and want to stick to that, you will skip the step of identifying the best fit distribution.
#scipy does not come with a function to calculate log likelihood (although MLE method is provided), but hard code one is easy, see:
#https://stackoverflow.com/questions/18431629/is-the-build-in-probability-density-functions-of-scipy-stat-distributions-slow

# use half of dataset (randomly drawn) to find distribution params utilizing MLE
x_Delay = X_train_Delay_ORDDOOGE

#fit exponential dist
best_fit_Delay_params = [1.0, 11.053763440860216] #best fit pdf computed above is diff from expon, set params and use expon anyway
params_Delay_exp = best_fit_Delay_params #for this example, take initial params from best fit pdf function call using SSE above
params_Delay_exp_SSE = best_fit_Delay_params 
y_Delay = st.expon.pdf(x_Delay)
    
#fit gamma dist
#params_gamma = scp.stats.gamma.fit(X_train_Delay_ORDDOOGE, floc=0)
#y = scp.stats.gamma.pdf(x, params_gamma)

#fit lognorm dist
#params_lognorm = scp.stats.lognorm.fit(X_train_Delay_ORDDOOGE, floc=0)
#y = st.lognorm.pdf(x, params_lognorm)

#fit weib dist
#params_Delay_weib = st.weibull_min.fit(x_Delay.iloc[:,0], floc=0, f0=1) #have to convert dataframe to series
#y = st.weibull_min.pdf(x_Delay, best_fit_Delay_params)

# plot y against x
#sns.regplot(x_Delay, y_Delay, fit_reg=False)

# fit OLS model and summarize
#OLS_results = sm.OLS(y,x).fit()
#print(OLS_results.summary())
#print('OLS Regression Parameters: ', OLS_results.params)

# define likelihood function
def MLERegression_Delay(params):
    # inputs are guesses at our parameters
    loc_init, sd = params[0], params[1] #use for expon, halfnorm pdf
    
    #use for gamma, lognorm, exponnorm (where K is alpha), weibull pdf
    #alpha, loc_init, sd = params[0], params[1], params[2] 
    
    #Note that a prediction value is the mean of a distribution
    #We often think of a prediction as a single scalar (or vector) value 
    #that the model predicts will be the output from a given input. 
    #In fact, the prediction from a regression model is a probability distribution on the values that could be output.
    #The single prediction we are used to seeing is just the mean of that distribution.
    #See: https://www.cs.cmu.edu/~schneide/tut5/node31.html
    yhat = st.expon.pdf(loc_init)
    #yhat = scp.stats.gamma.pdf(loc_init, alpha)
    #yhat = st.lognorm.pdf(loc_init, alpha)
    #yhat = st.weibull_min.pdf(loc_init, alpha)
    
# next, we flip the Bayesian question
# compute PDF of observed values normally distributed around mean (yhat)
# with a standard deviation of sd - because central limit theorem.
    negLL = -np.sum( st.norm.logpdf(y_Delay, loc=yhat, scale=sd) )
# return negative LL
    return(negLL)

# let’s start with initial coefficients computed above and optimize
# to learn more: https://scipy-lectures.org/advanced/mathematical_optimization/
results_Delay = minimize(MLERegression_Delay, params_Delay_exp, method='Nelder-Mead', options={'disp': True})
MLE_best_fit_Delay_params = results_Delay.x

##################################################################
# use half of dataset (randomly drawn) to find distribution params utilizing MLE
x_mfd = X_train_meterFixDistance_ORDDOOGE

#fit exponential dist
#params_exp = best_fit_params
#y_mfd = st.expon.pdf(x_mfd)

#fit gamma dist
#params_gamma = scp.stats.gamma.fit(X_train_Delay_ORDDOOGE, floc=0)
#y_mfd = scp.stats.gamma.pdf(x_mfd, params_gamma)

#fit lognorm dist
#params_mfd_lognorm = st.lognorm.fit(X_train_meterFixDistance_ORDDOOGE, floc=0)
params_mfd_lognorm = best_fit_mfd_params #for this example, take initial params from best fit pdf function call using SSE above
params_mfd_lognorm_SSE = best_fit_mfd_params
y_mfd = st.lognorm.pdf(x_mfd, params_mfd_lognorm)

#fit halfnorm dist
#params_mfd_halfnorm = st.halfnorm.fit(x_mfd)
#y_mfd = st.halfnorm.pdf(x_mfd)

#fit exponnorm dist
#params_mfd_exponnorm = st.exponnorm.fit(x_mfd, floc=0)
#y_mfd = st.exponnorm.pdf(pd.DataFrame(x_mfd), best_fit_mfd_params)

#fit weib dist
#params_weib = st.weibull_min.fit(x.iloc[:,0], floc=0, f0=1) #have to convert dataframe to series
#y_mfd = st.weibull_min.pdf(x_mfd, params_weib)

# define likelihood function
def MLERegression_mfd(params):
    # inputs are guesses at our parameters
    #loc_init, sd = params[0], params[1] #use for expon, halfnorm pdf
    
    #use for gamma, lognorm, exponnorm (where K is alpha), weibull pdf
    alpha, loc_init, sd = params[0], params[1], params[2]
    
    #Note that a prediction value is the mean of a distribution
    #We often think of a prediction as a single scalar (or vector) value 
    #that the model predicts will be the output from a given input. 
    #In fact, the prediction from a regression model is a probability distribution on the values that could be output.
    #The single prediction we are used to seeing is just the mean of that distribution.
    #See: https://www.cs.cmu.edu/~schneide/tut5/node31.html
    #yhat = scp.stats.expon.pdf(loc_init)
    #yhat = scp.stats.halfnorm.pdf(loc_init)
    #yhat = scp.stats.gamma.pdf(loc_init, alpha)
    yhat = st.lognorm.pdf(loc_init, alpha)
    #yhat = st.exponnorm.pdf(loc_init, alpha)
    #yhat = st.weibull_min.pdf(loc_init, alpha)
    
# next, we flip the Bayesian question
# compute PDF of observed values normally distributed around mean (yhat)
# with a standard deviation of sd - because central limit theorem
    negLL = -np.sum( st.norm.logpdf(y_mfd, loc=yhat, scale=sd) )
# return negative LL
    return(negLL)

# let’s start with initial coefficients computed above and optimize
# to learn more: https://scipy-lectures.org/advanced/mathematical_optimization/
results_mfd = minimize(MLERegression_mfd, params_mfd_lognorm, method='Nelder-Mead', options={'disp': True})
MLE_best_fit_mfd_params = results_mfd.x


Optimization terminated successfully.
         Current function value: -2315.645375
         Iterations: 62
         Function evaluations: 118


/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:899: RuntimeWarning: invalid value encountered in greater_equal
  return (a <= x) & (x <= b)
/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:899: RuntimeWarning: invalid value encountered in less_equal
  return (a <= x) & (x <= b)


In [16]:
#########################################################################
#Run statistical test to assess goodness-of-fit for the ORD_DOOGE delay distribution 
#The train datatest was used to find distribution parameters.
#The KS test is performed on the other half (the test set) of ORDDOOGE delay dataset
#You can also use Lilliefors if you test against an exp or norm dist.
#The advantage of using Lilliefors is that you test against the whole set of data, instead of having
#to split the data into train vs. test set, since Lilliefors will also estimate the params for the distribution
#########################################################################

#D'sub.05 = 1.36/sqrt(n) where n = 15302/2: D = .016
stat_Delay, p_Delay = st.kstest(X_test_Delay_ORDDOOGE, 'expon', args=(MLE_best_fit_Delay_params), alternative='two-sided')
print("KS test results for Delay dataset using MLE Regression best fit params: D = ", stat_Delay, "p = ", p_Delay)
print ("MLE best fit Delay params: ", MLE_best_fit_Delay_params)

#Compute SSE using MLE best fit Delay params
y, x = np.histogram(X_test_Delay_ORDDOOGE, bins=200, density=True)
x = (x + np.roll(x, -1))[:-1] / 2.0
arg = MLE_best_fit_Delay_params[:-2]
loc = MLE_best_fit_Delay_params[-2]
scale = MLE_best_fit_Delay_params[-1]
pdf = st.expon.pdf(x, loc=loc, scale=scale, *arg)
sse = np.sum(np.power(y - pdf, 2.0))
print("Compute SSE using MLE best fit Delay params: ", sse)

stat_Delay, p_Delay = st.kstest(X_test_Delay_ORDDOOGE, 'expon', args=(params_Delay_exp_SSE), alternative='two-sided')
print("KS test results for Delay using best fit params w/ SSE compare method: D = ", stat_Delay, "p = ", p_Delay)
print ("Best fit Delay params by SSE compare method: ", best_fit_Delay_params)

#Run lilliefors test, a Kolmogorov-Smirnov test with estimated parameters, only use for normal and exp dist
print("Use Lilliefors test results if tested against exp or norm dist: ", lilliefors(subset_Delay_ORDDOOGE_noO, dist='exp', pvalmethod='table'))


print("\n")
#################################################################
#Run KS test on the other half (the test set) of the ORDDOOGE meterFixDistance dataset
################################################################

#D'sub.05 = 1.36/sqrt(n) where n = 15302/2: D = .016
stat_mfd, p_mfd = st.kstest(X_test_meterFixDistance_ORDDOOGE, 'lognorm', args=(MLE_best_fit_mfd_params), alternative='two-sided')
print("KS test results for meterFixDistance data using MLE Regression best fit params: D = ", stat_Delay, "p = ", p_Delay)
print ("MLE best fit mfd params: ", MLE_best_fit_mfd_params)

#Compute SSE using MLE best fit mfd params
y, x = np.histogram(X_test_meterFixDistance_ORDDOOGE, bins=200, density=True)
x = (x + np.roll(x, -1))[:-1] / 2.0
arg = MLE_best_fit_mfd_params[:-2]
loc = MLE_best_fit_mfd_params[-2]
scale = MLE_best_fit_mfd_params[-1]
pdf = st.lognorm.pdf(x, loc=loc, scale=scale, *arg)
sse = np.sum(np.power(y - pdf, 2.0))
print("Compute SSE using MLE best fit mfd params: ", sse)

stat_mfd, p_mfd = st.kstest(X_test_meterFixDistance_ORDDOOGE, 'lognorm', args=(params_mfd_lognorm_SSE), alternative='two-sided')
print("KS test results for meterFixDistance data using best fit params w/ SSE compare method: D = ", stat_mfd, "p = ", p_mfd)
print ("Best fit mfd params by SSE compare method: ", params_mfd_lognorm_SSE)

#Run lilliefors test, a Kolmogorov-Smirnov test with estimated parameters, only use for normal and exp dist
print("Use Lilliefors test results if tested against exp or norm dist: ", lilliefors(subset_meterFixDistance_ORDDOOGE_noO, dist='norm', pvalmethod='table'))


KS test results for Delay dataset using MLE Regression best fit params: D =  1.0 p =  0.0
MLE best fit Delay params:  [3.46629421 0.08195949]
Compute SSE using MLE best fit Delay params:  5.7882259860622325
KS test results for Delay using best fit params w/ SSE compare method: D =  1.0 p =  0.0
Best fit Delay params by SSE compare method:  [1.0, 11.053763440860216]
Use Lilliefors test results if tested against exp or norm dist:  (0.9561994707795167, 0.0009999999999998899)


KS test results for meterFixDistance data using MLE Regression best fit params: D =  1.0 p =  0.0
MLE best fit mfd params:  [  0.27759256   0.         161.60004452]
Compute SSE using MLE best fit mfd params:  0.09241968854248106
KS test results for meterFixDistance data using best fit params w/ SSE compare method: D =  0.9937331698745766 p =  0.0
Best fit mfd params by SSE compare method:  (0.2775925605454191, 0.0, 161.60004452005663)
Use Lilliefors test results if tested against exp or norm dist:  (0.99960298594717

In [17]:
#########################################################
#Normalize Delay data using scaling and power transform #
#########################################################

pdf = matplotlib.backends.backend_pdf.PdfPages("ORDDOOGE_PP_Delay_plots.pdf")

X_train_Delay_ORDDOOGE_xformed = xform(X_train_Delay_ORDDOOGE)
X_test_Delay_ORDDOOGE_xformed = xform(X_test_Delay_ORDDOOGE)

left = -1.8   #x coordinate for text insert

fig = plt.figure()

ax = fig.add_subplot(221)
prob = st.probplot(X_train_Delay_ORDDOOGE_xformed.flatten(), dist=st.norm, plot=ax)
ax.set_title('PP Plot: ORD_DOOGE delays data against a normal dist')
top = ax.get_ylim()[1] * 0.75
txt = ax.text(left, top, 'Data is normalized using Box-Cox transformation', verticalalignment='top')
txt.set_bbox(dict(facecolor='k', alpha=0.1))

ax = fig.add_subplot(222)
prob = plt.hist(X_train_Delay_ORDDOOGE_xformed)
ax.set_title('Histogram of normalized ORD_DOOGE delays data')

ax = fig.add_subplot(223)
prob = st.probplot(X_test_Delay_ORDDOOGE_xformed.flatten(), dist=st.norm, plot=ax)
ax.set_title('PP Plot: ORD_DOOGE delays data against a normal dist')
top = ax.get_ylim()[1] * 0.75
txt = ax.text(left, top, 'Data is normalized using Box-Cox transformation', verticalalignment='top')
txt.set_bbox(dict(facecolor='k', alpha=0.1))

ax = fig.add_subplot(224)
prob = plt.hist(X_test_Delay_ORDDOOGE_xformed)
ax.set_title('Histogram of normalized ORD_DOOGE delays data')

fig.tight_layout()
pdf.savefig()

#Run KS test against a normal dist on test dataset using params derived from train dataset
s = pd.DataFrame(X_train_Delay_ORDDOOGE_xformed).describe(include='all')
print (s)
X_train_Delay_ORDDOOGE_fit_params = [s.loc['mean'], s.loc['std']]
statTest, pTest = st.kstest(X_test_Delay_ORDDOOGE_xformed, 'norm', args=(X_train_Delay_ORDDOOGE_fit_params))
sizeX_test = len(X_test_Delay_ORDDOOGE_xformed)
print ("D'sub.05 for ORD_DOOGE delay data w/ sample size of", sizeX_test, " = ", 1.36/np.sqrt(sizeX_test))
print("KS test results against normal dist: D = ", statTest, "p = ", pTest)

plt.close('all')
pdf.close()

                  0
count  2.139000e+03
mean   4.766848e-16
std    1.000234e+00
min   -1.959802e+00
25%   -6.815900e-01
50%    6.206125e-02
75%    8.098156e-01
max    1.901708e+00
D'sub.05 for ORD_DOOGE delay data w/ sample size of 2140  =  0.029398963679363954
KS test results against normal dist: D =  0.9785301532973321 p =  0.0


In [18]:
####################################################################
#normalize meterFixDistance data using scaling and power transform #
####################################################################

pdf = matplotlib.backends.backend_pdf.PdfPages("ORDDOOGE_PP_mfd_plots.pdf")

X_train_meterFixDistance_ORDDOOGE_xformed = xform(X_train_meterFixDistance_ORDDOOGE)
X_test_meterFixDistance_ORDDOOGE_xformed = xform(X_test_meterFixDistance_ORDDOOGE)

left = -1.8   #x coordinate for text insert

fig = plt.figure()

ax = fig.add_subplot(221)
prob = st.probplot(X_train_meterFixDistance_ORDDOOGE_xformed.flatten(), dist=st.norm, plot=ax)
ax.set_title('PP Plot: distance to the meter fix ORD_DOOGE data against a normal dist')
top = ax.get_ylim()[1] * 0.75
txt = ax.text(left, top, 'Data is normalized using Box-Cox transformation', verticalalignment='top')
txt.set_bbox(dict(facecolor='k', alpha=0.1))

ax = fig.add_subplot(222)
prob = plt.hist(X_train_meterFixDistance_ORDDOOGE_xformed)
ax.set_title('Histogram of normalized distance to the meter fix ORD_DOOGE data')

ax = fig.add_subplot(223)
prob = st.probplot(X_test_meterFixDistance_ORDDOOGE_xformed.flatten(), dist=st.norm, plot=ax)
ax.set_title('PP Plot: distance to the meter fix ORD_DOOGE data against a normal dist')
top = ax.get_ylim()[1] * 0.75
txt = ax.text(left, top, 'Data is normalized using Box-Cox transformation', verticalalignment='top')
txt.set_bbox(dict(facecolor='k', alpha=0.1))

ax = fig.add_subplot(224)
prob = plt.hist(X_test_meterFixDistance_ORDDOOGE_xformed)
ax.set_title('Histogram of normalized distance to the meter fix ORD_DOOGE data')
fig.tight_layout()

pdf.savefig()

#Run KS test against a normal dist on test dataset using params derived from train dataset
s = pd.DataFrame(X_train_meterFixDistance_ORDDOOGE_xformed).describe(include='all')
print (s)
X_train_meterFixDistance_ORDDOOGE_fit_params = [s.loc['mean'], s.loc['std']]
statTest_mfd, pTest_mfd = st.kstest(X_test_meterFixDistance_ORDDOOGE_xformed, 'norm', args=(X_train_meterFixDistance_ORDDOOGE_fit_params))
sizeX_mfd_test = len(X_test_meterFixDistance_ORDDOOGE_xformed)
print ("D'sub.05 for distance to the meter fix ORD_DOOGE data w/ sample size of", sizeX_mfd_test, " = ", 1.36/np.sqrt(sizeX_mfd_test))
print("KS test results against normal dist: D = ", statTest_mfd, "p = ", pTest_mfd)

plt.close('all')
pdf.close()

                  0
count  2.188000e+03
mean  -1.280308e-15
std    1.000229e+00
min   -2.146821e+00
25%   -6.476874e-01
50%   -4.074602e-01
75%    9.777444e-01
max    2.573327e+00
D'sub.05 for distance to the meter fix ORD_DOOGE data w/ sample size of 2189  =  0.029068058636443742
KS test results against normal dist: D =  0.9960206175150087 p =  0.0


In [19]:
################################################
#format & transform data for linear regression #
################################################


#To retain the integrity of [distance, delay] pairs, first we create tuples
subsetReg_ORDDOOGE = []
for x,y in zip(subset_ORDDOOGE['meterFixDistance'], subset_ORDDOOGE['FinalDelay']):
    subsetReg_ORDDOOGE.append([x,y])
print("Dataset initial size: ", len(subsetReg_ORDDOOGE))


# remove outliers for delays or y using IQR
# calculate interquartile range
q25_delay, q75_delay = np.percentile(subset_ORDDOOGE['FinalDelay'], 25), np.percentile(subset_ORDDOOGE['FinalDelay'], 75)
iqr_delay = q75_delay - q25_delay
print('Percentiles for dataset: 25th=%.3f, 75th=%.3f, IQR=%.3f' % (q25_delay, q75_delay, iqr_delay))
# calculate the outlier cutoff
cut_off_delay = iqr_delay * 1.5
lower_delay, upper_delay = q25_delay - cut_off_delay, q75_delay + cut_off_delay

subsetReg_ORDDOOGE_noO = [pair for pair in subsetReg_ORDDOOGE if pair[1] >= lower_delay and pair[1] <= upper_delay]
print('Non-outlier observations for ORD_DOOGE delay: %d' % len(subsetReg_ORDDOOGE_noO))


# remove outliers for mfd or x using IQR
# calculate interquartile range
q25_mfd, q75_mfd = np.percentile(subset_ORDDOOGE['meterFixDistance'], 25), np.percentile(subset_ORDDOOGE['meterFixDistance'], 75)
iqr_mfd = q75_mfd - q25_mfd
print('Percentiles for dataset: 25th=%.3f, 75th=%.3f, IQR=%.3f' % (q25_mfd, q75_mfd, iqr_mfd))
# calculate the outlier cutoff
cut_off_mfd = iqr_mfd * 1.5
lower_mfd, upper_mfd = q25_mfd - cut_off_mfd, q75_mfd + cut_off_mfd

subsetReg_ORDDOOGE_noO = [pair for pair in subsetReg_ORDDOOGE if pair[0] >= lower_mfd and pair[0] <= upper_mfd]
print('Non-outlier observations for meterFixDistance: %d' % len(subsetReg_ORDDOOGE_noO))


#Then transform the tuple into a list to perform normalization
subsetReg_ORDDOOGE = []
subsetReg_woO_ORDDOOGE = [[pair[0] for pair in subsetReg_ORDDOOGE_noO], [pair[1] for pair in subsetReg_ORDDOOGE_noO]]

#normalize meterFixDistance data using scaling
max_abs_scaler = preprocessing.MaxAbsScaler()
subsetReg_ORDDOOGE_normalized = [max_abs_scaler.fit_transform(pd.DataFrame(subsetReg_woO_ORDDOOGE[0])), subsetReg_woO_ORDDOOGE[1]]

#Normalize Delay data using scaling & Yeo-Johnson
mm_scaler = preprocessing.MinMaxScaler()
subsetReg_ORDDOOGE_normalized = [subsetReg_ORDDOOGE_normalized[0], mm_scaler.fit_transform(pd.DataFrame(subsetReg_woO_ORDDOOGE[1]))]

X_Reg_ORDDOOGE = preprocessing.power_transform(subsetReg_ORDDOOGE_normalized[0], method='yeo-johnson')
Y_Reg_ORDDOOGE = preprocessing.power_transform(subsetReg_ORDDOOGE_normalized[1], method='yeo-johnson')

#If you want to use box-cox transformation, you need to transform the list to tuples again to retain
#only positive data, or skip the scaler (see the xform function call)
#subsetReg_ORDDOOGE = []
#for x,y in zip(subsetReg_ORDDOOGE_normalized[0], subsetReg_ORDDOOGE_normalized[1]):
#    subsetReg_ORDDOOGE.append([x,y])

#Keep stricly positive data only
#subsetReg_ORDDOOGE_keep_pos = [pair for pair in subsetReg_ORDDOOGE if pair[0] > 0 and pair[1] > 0]

#Transform the tuples into a list
#subsetReg_ORDDOOGE = []
#subsetReg_ORDDOOGE = [[float(pair[0]) for pair in subsetReg_ORDDOOGE_keep_pos], [float(pair[1]) for pair in subsetReg_ORDDOOGE_keep_pos]]

#Run boxcox power transform
#X_Reg_ORDDOOGE = st.boxcox(subsetReg_ORDDOOGE[0], 1)
#Y_Reg_ORDDOOGE = st.boxcox(subsetReg_ORDDOOGE[1], 0)

#df = pd.DataFrame({'mfd': X_Reg_ORDDOOGE, 'delay': Y_Reg_ORDDOOGE})
#print(df)

Dataset initial size:  4433
Percentiles for dataset: 25th=5.000, 75th=18.000, IQR=13.000
Non-outlier observations for ORD_DOOGE delay: 4279
Percentiles for dataset: 25th=136.039, 75th=211.839, IQR=75.800
Non-outlier observations for meterFixDistance: 4377


In [20]:
##################################################
#Fit linear regression model to transformed data #
#Compute Pearson coefficient                     #
##################################################

#########################################
# Linear regression on normalized data

print('Fit linear regression model on normalized dataset')
X_train_Reg_ORDDOOGE, X_test_Reg_ORDDOOGE, Y_train_Reg_ORDDOOGE, Y_test_Reg_ORDDOOGE = train_test_split(X_Reg_ORDDOOGE.flatten(), Y_Reg_ORDDOOGE.flatten(), test_size=.5)

# define and fit the model on all data
model = LinearRegression()
model.fit(pd.DataFrame(X_train_Reg_ORDDOOGE), pd.DataFrame(Y_train_Reg_ORDDOOGE))

#To retrieve the intercept:
print("intercept: ", model.intercept_)
#For retrieving the slope:
print("slope: ", model.coef_)

# number of parameters
num_params = len(model.coef_) + 1
#print('Number of parameters: %d' % (num_params))

# predict the training set
yhat = model.predict(pd.DataFrame(X_test_Reg_ORDDOOGE))

#df = pd.DataFrame({'Actual': Y_test_Reg_ORDDOOGE.to_numpy().flatten(), 'Predicted': yhat.flatten()})
#print(df)
print('Statistics for ORD_DOOGE delays - normalized test set')
#print(pd.DataFrame(Y_test_Reg_ORDDOOGE).describe())

# calculate the error
mse = mean_squared_error(Y_test_Reg_ORDDOOGE, yhat)
print('MSE: %.3f' % mse)
rmse = np.sqrt(mse)
print('RMSE: %.3f' % rmse)
# calculate the aic
aic = calculate_aic(len(Y_test_Reg_ORDDOOGE), mse, num_params)
print('AIC: %.3f' % aic)
# calculate the bic
bic = calculate_bic(len(Y_test_Reg_ORDDOOGE), mse, num_params)
print('BIC: %.3f' % bic)

#compute the pearson coefficient
#to learn more: https://machinelearningmastery.com/how-to-use-correlation-to-understand-the-relationship-between-variables/
print("Pearson Coefficient for ORD_DOOGE delays -  normalized test set:\n", st.pearsonr(X_test_Reg_ORDDOOGE, Y_test_Reg_ORDDOOGE))
print(st.linregress(X_test_Reg_ORDDOOGE, Y_test_Reg_ORDDOOGE))

##################################################
# Linear regression on raw data, outliers removed

print('\nFit linear regression model on raw dataset, outliers removed')
print('However, note that linear regression works best on normalized data')
X_train_Reg_ORDDOOGE_raw, X_test_Reg_ORDDOOGE_raw, Y_train_Reg_ORDDOOGE_raw, Y_test_Reg_ORDDOOGE_raw = train_test_split(subsetReg_woO_ORDDOOGE[0], subsetReg_woO_ORDDOOGE[1], test_size=.5)

model.fit(pd.DataFrame(X_train_Reg_ORDDOOGE_raw), pd.DataFrame(Y_train_Reg_ORDDOOGE_raw))

#To retrieve the intercept:
print("intercept: ", model.intercept_)
#For retrieving the slope:
print("slope: ", model.coef_)

# number of parameters
num_params_raw = len(model.coef_) + 1
#print('Number of parameters: %d' % (num_params))

# predict the training set
yhat_raw = model.predict(pd.DataFrame(X_test_Reg_ORDDOOGE_raw))

#df = pd.DataFrame({'Actual': Y_test_Reg_ORDDOOGE.to_numpy().flatten(), 'Predicted': yhat.flatten()})
#print(df)

print('Statistics for ORD_DOOGE delays - raw test set')
#print(pd.DataFrame(Y_test_Reg_ORDDOOGE_raw).describe())

# calculate the error
mse_raw = mean_squared_error(Y_test_Reg_ORDDOOGE_raw, yhat_raw)
print('MSE: %.3f' % mse_raw)
rmse_raw = np.sqrt(mse_raw)
print('RMSE: %.3f' % rmse_raw)
# calculate the aic
aic_raw = calculate_aic(len(Y_test_Reg_ORDDOOGE_raw), mse_raw, num_params_raw)
print('AIC: %.3f' % aic_raw)
# calculate the bic
bic_raw = calculate_bic(len(Y_test_Reg_ORDDOOGE_raw), mse_raw, num_params_raw)
print('BIC: %.3f' % bic_raw)

#Pearson coefficient
print("Note that the Pearson Coefficient works strictly on normalized data.")
print("However, here's the Pearson Coefficient for ORD_DOOGE delays on raw test set:\n", st.pearsonr(X_test_Reg_ORDDOOGE_raw, Y_test_Reg_ORDDOOGE_raw))
print(st.linregress(X_test_Reg_ORDDOOGE_raw, Y_test_Reg_ORDDOOGE_raw))


Fit linear regression model on normalized dataset
intercept:  [-0.00159799]
slope:  [[-0.08499588]]
Statistics for ORD_DOOGE delays - normalized test set
MSE: 0.986
RMSE: 0.993
AIC: -26.181
BIC: -14.798
Pearson Coefficient for ORD_DOOGE delays -  normalized test set:
 (-0.09136353074898425, 1.8595736336258186e-05)
LinregressResult(slope=-0.09040896350383074, intercept=0.0015482686919906148, rvalue=-0.09136353074898423, pvalue=1.8595736336204037e-05, stderr=0.021071430725717863)

Fit linear regression model on raw dataset, outliers removed
However, note that linear regression works best on normalized data
intercept:  [17.83999279]
slope:  [[-0.02853896]]
Statistics for ORD_DOOGE delays - raw test set
MSE: 114.789
RMSE: 10.714
AIC: 10386.627
BIC: 10398.009
Note that the Pearson Coefficient works strictly on normalized data.
However, here's the Pearson Coefficient for ORD_DOOGE delays on raw test set:
 (-0.0846093955132574, 7.38755720853427e-05)
LinregressResult(slope=-0.01973437766366937

In [21]:
##################################################
#Scatter plot for raw data w/o outliers          #
##################################################

sns.set(style="ticks", color_codes=True)
pdf = matplotlib.backends.backend_pdf.PdfPages("ORDDOOGE_scatterPlots_noOutliers.pdf")


Reg_ORDDOOGE_raw=pd.DataFrame(list(zip(subsetReg_woO_ORDDOOGE[0], subsetReg_woO_ORDDOOGE[1])),
                            columns=['meterFixDistance', 'Delay'])

ax1 = sns.scatterplot(Reg_ORDDOOGE_raw['meterFixDistance'], Reg_ORDDOOGE_raw['Delay'], palette='GnBu_r',
                  hue=Reg_ORDDOOGE_raw['Delay'], size=Reg_ORDDOOGE_raw['meterFixDistance'], linewidth=1, alpha = 1)

ax1.set_title('Scatter plot for ORD_DOOGE - raw data w/o outliers')
ax1.set_xlabel('Meter Fix Distance (nm)')
ax1.set_ylabel('Delay (minutes)')
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)

pdf.savefig()
plt.close('all')
pdf.close()

In [22]:
##################################################
#Scatter plot for normalized data                #
##################################################

pdf = matplotlib.backends.backend_pdf.PdfPages("ORDDOOGE_scatterPlots_normalized.pdf")
sns.set(style="ticks", color_codes=True)


Reg_ORDDOOGE=pd.DataFrame(list(zip(X_Reg_ORDDOOGE.flatten(), Y_Reg_ORDDOOGE.flatten())),
                            columns=['meterFixDistance', 'Delay'])

ax1 = sns.scatterplot(Reg_ORDDOOGE['meterFixDistance'], Reg_ORDDOOGE['Delay'], palette='GnBu_r',
                  hue=Reg_ORDDOOGE['Delay'], size=Reg_ORDDOOGE['meterFixDistance'], linewidth=1, alpha = 1)

ax1.set_title('Scatter plot for ORD_DOOGE - normalized data')
ax1.set_xlabel('Meter Fix Distance (nm)')
ax1.set_ylabel('Delay (minutes)')
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)

pdf.savefig()
plt.close('all')
pdf.close()

SOME REMARKS FOR THE SECOND QUESTION:<br />
Here, eventhough we did not pass normality tests for both delay and meterFixDistance, we went ahead and perform a linear regression anyway. We did not find a linear relationship between these 2 datasets. At this point, we should assess for additional features that could correlate with the delay. For example:<br />
- time of day/week/year<br />
- <br />  

We can statistically assess 2-3 features for correlation with our target variable.  Using ANOVA, we can assess the features at several different levels for each feature. We can also perform multiple regression.  Statistical methodologies beyond 3 features start to become more and more burdensome however, and here is where we hope machine-learning techniques can pick up the baton.<br />